## Importing libraries

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor
import pandas as pd
import html
import time
import csv
# from scrapy.downloadermiddlewares.retry import RetryMiddleware
# from scrapy_random_proxies.middlewares import RandomProxyMiddleware
import requests
from bs4 import BeautifulSoup
import random
from requests_ip_rotator import ApiGateway, EXTRA_REGIONS
import winsound
import os
import datetime
import re
from math import ceil
import re

## Datasets

#### First of all, we need to collect the data from CodeForces website
##### we will do this after simulating a login session

In [27]:
# Obtain the CSRF token from the login page
login_url = 'https://codeforces.com/enter'
session = requests.Session()
response = session.get(login_url)
soup = BeautifulSoup(response.content, 'html.parser')
csrf_token = soup.find('input', {'name': 'csrf_token'}).get('value')
# Specify your login credentials
username = 'roro_1999_hh@hotmail.com'
password = 'Dodehtree*-*19781987'

# Send a POST request with the login credentials and CSRF token
login_data = {
    'csrf_token': csrf_token,
    'action': 'enter',
    'handleOrEmail': username,
    'password': password,
    'remember': 'on'  # You can remove this line if you don't want to be remembered
}

session.post(login_url, data=login_data)


<Response [200]>

### 1) Problemset

#### Problemset of the first page in problemset in codeforces

In [4]:
problemset_url = 'https://codeforces.com/problemset?order=BY_SOLVED_DESC'

# Send a GET request to the URL
response = requests.get(problemset_url)

# Get the HTML content
html_content = response.content

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table containing the problemset
table = soup.find('table', class_='problems')

# Extract the desired columns
rows = table.find_all('tr')
problemset_data = []
header = ['#', 'Name', 'Difficulty', 'Solved']

for row in rows[1:]:
    cells = row.find_all('td')
    num = cells[0].text.strip()
    name = cells[1].text.strip()
    difficulty = cells[3].text.strip()
    solved = cells[4].text.strip()
    
    problemset_data.append([num, name, difficulty, solved])

# Create a DataFrame from the extracted data
problemset_df = pd.DataFrame(problemset_data, columns=header)

# Print the DataFrame
problemset_df

,#,Name,Difficulty,Solved
0,4A,Watermelon \n\n\nbrute forc...,800,x394103
1,71A,Way Too Long Words \n\n\nst...,800,x289869
2,231A,"Team \n\n\nbrute force,\r\n...",800,x243993
3,1A,Theatre Square \n\n\nmath,1000,x210189
4,158A,Next Round \n\n\n*special p...,800,x194010
...,...,...,...,...
95,579A,Raising Bacteria \n\n\nbitm...,1000,x46248
96,313A,Ilya and Bank Account \n\n\...,900,x46076
97,1742A,Sum \n\n\nimplementation,800,x45721
98,703A,Mishka and Game \n\n\nimple...,800,x45563


### Preprocessing Problemset dataframe

###### First,
1. Split the values in the '#' column

In [163]:
# Split the values in the '#' column
problemset_df['Number'] = problemset_df['#'].str[:-1]
problemset_df['Character'] = problemset_df['#'].str[-1]

# Print the modified DataFrame
problemset_df

,#,Name,Difficulty,Solved,Number,Character
0,4A,Watermelon \n\n\nbrute forc...,800,x394103,4,A
1,71A,Way Too Long Words \n\n\nst...,800,x289869,71,A
2,231A,"Team \n\n\nbrute force,\r\n...",800,x243993,231,A
3,1A,Theatre Square \n\n\nmath,1000,x210189,1,A
4,158A,Next Round \n\n\n*special p...,800,x194010,158,A
...,...,...,...,...,...,...
95,579A,Raising Bacteria \n\n\nbitm...,1000,x46248,579,A
96,313A,Ilya and Bank Account \n\n\...,900,x46076,313,A
97,1742A,Sum \n\n\nimplementation,800,x45721,1742,A
98,703A,Mishka and Game \n\n\nimple...,800,x45563,703,A


###### Second,
2. Preprocessing Solved column

In [178]:
problemset_df['x'] = problemset_df['Solved'].str[1]
problemset_df['Solved'] = problemset_df['Solved'].str[1:]
problemset_df = problemset_df.drop(columns= ['x'])
# Print the modified DataFrame
problemset_df

,#,Name,Difficulty,Solved,Number,Character
0,4A,Watermelon \n\n\nbrute forc...,800,394103,4,A
1,71A,Way Too Long Words \n\n\nst...,800,289869,71,A
2,231A,"Team \n\n\nbrute force,\r\n...",800,243993,231,A
3,1A,Theatre Square \n\n\nmath,1000,210189,1,A
4,158A,Next Round \n\n\n*special p...,800,194010,158,A
...,...,...,...,...,...,...
95,579A,Raising Bacteria \n\n\nbitm...,1000,46248,579,A
96,313A,Ilya and Bank Account \n\n\...,900,46076,313,A
97,1742A,Sum \n\n\nimplementation,800,45721,1742,A
98,703A,Mishka and Game \n\n\nimple...,800,45563,703,A


###### Third,
3. Preprocessing Name column

In [179]:
# Split the Name column when a newline character is encountered
problemset_df[['Problem Name', 'Tag']] = problemset_df['Name'].str.split('\n\n\n', expand=True)
# Drop the original Name column
problemset_df.drop('Name', axis=1, inplace=True)

# Print the modified DataFrame
problemset_df

,#,Difficulty,Solved,Number,Character,Problem Name,Tag
0,4A,800,394103,4,A,Watermelon,"brute force,\r\n math"
1,71A,800,289869,71,A,Way Too Long Words,strings
2,231A,800,243993,231,A,Team,"brute force,\r\n greedy"
3,1A,1000,210189,1,A,Theatre Square,math
4,158A,800,194010,158,A,Next Round,"*special problem,\r\n i..."
...,...,...,...,...,...,...,...
95,579A,1000,46248,579,A,Raising Bacteria,bitmasks
96,313A,900,46076,313,A,Ilya and Bank Account,"implementation,\r\n num..."
97,1742A,800,45721,1742,A,Sum,implementation
98,703A,800,45563,703,A,Mishka and Game,implementation


###### Fourth,
4. Renaming Character column to Class

In [781]:
problemset_df.rename(columns={'Character': 'Class'}, inplace=True)

problemset_df.rename(columns={'#': 'Number_Class'}, inplace=True)


###### Fifth,
5. First 50 problems in problemset
#### We will keep only the submissions on the first 50 problems so that we can effectively scrape later on

In [13]:
problemset_50_df = problemset_df.head(50)
problemset_50_df['Number'] = problemset_50_df['Number'].astype(str)
problemset_50_df['Class'] = problemset_50_df['Class'].astype(str)

C:\Users\dralh\AppData\Local\Temp\ipykernel_8012\3697334036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problemset_50_df['Number'] = problemset_50_df['Number'].astype(str)
C:\Users\dralh\AppData\Local\Temp\ipykernel_8012\3697334036.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problemset_50_df['Class'] = problemset_50_df['Class'].astype(str)


In [2]:
# problemset_df.to_csv("Processed_problemset_df.csv", index = False)
problemset_df = pd.read_csv("Processed_problemset_df.csv")

## Features engineering

###### First,
1. Preprocessing Tag column
   1) Getting rid of extra characters
2. Feature's name: problem_tag
   Splitting into multiple tags columns that has either 0 or 1

In [5]:
problemset_df['Tag'] = problemset_df['Tag'].apply(lambda x: str(x).replace('\r\n', ''))


column = problemset_df['Tag']

keep_symbol = ','

for i in range(len(column)):
    symbols = [symbol for symbol in column[i] if symbol == keep_symbol or symbol.isalpha()]
    column[i] = ''.join(symbols)

problemset_df['Tag'] = column


In [6]:
# Split the tags column into separate columns
tags_df = problemset_df['Tag'].str.get_dummies(',').add_prefix('Tag_')

# Concatenate the tags DataFrame with the original DataFrame
problemset_df = pd.concat([problemset_df, tags_df], axis=1)


# Drop duplicate columns
problemset_df = problemset_df.loc[:,~problemset_df.columns.duplicated()]

# Drop the original Name and Tage columns
problemset_df.drop(['Tag'], axis=1, inplace=True)

# Print the modified DataFrame
problemset_df

,Number_Class,Difficulty,Solved,Number,Class,Problem Name,Tag_binarysearch,Tag_bitmasks,Tag_bruteforce,Tag_constructivealgorithms,...,Tag_greedy,Tag_hashing,Tag_implementation,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers
0,4A,800,394103,4,A,Watermelon,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,71A,800,289869,71,A,Way Too Long Words,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,231A,800,243993,231,A,Team,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,1A,1000,210189,1,A,Theatre Square,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,158A,800,194010,158,A,Next Round,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,579A,1000,46248,579,A,Raising Bacteria,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
96,313A,900,46076,313,A,Ilya and Bank Account,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
97,1742A,800,45721,1742,A,Sum,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
98,703A,800,45563,703,A,Mishka and Game,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
problemset_50_df = problemset_df.head(50)

### 2) Friends-Problemset Dataset

#### Scraping friends who solved the problems in the problemset of the first page in codeforces

In [360]:
base_url = 'https://codeforces.com/problemset/status/'

# 'https://codeforces.com/problemset/status/228/problem/A/page/'

header = ['#', 'When', 'Who', 'Problem', 'Language', 'Verdict', 'Time', 'Memory']   
rama = pd.DataFrame(columns=header)base_url = 'https://codeforces.com/problemset/status/'

# 'https://codeforces.com/problemset/status/228/problem/A/page/'

header = ['#', 'When', 'Who', 'Problem', 'Language', 'Verdict', 'Time', 'Memory']   
friends_problemset_df = pd.DataFrame(columns=header)


# Iterate over the values in 'Number' and 'Character' columns
for num, char in zip(problemset_df['Number'], problemset_df['Class']):
    
    # Access the protected page
    page = 1
    problem_url = f"{base_url}{num}/problem/{char}/page/"
    problem_page_url = f"{problem_url}{page}?friends=on"
    response = session.get(problem_page_url)
    soup = BeautifulSoup(response.content, 'lxml')
    s = soup.find('div', id= 'body')
    
    # Extract the total number of pages
    pagination_div = s.find('div', style='margin-top:1em;' ).find('div',class_= 'pagination')
    if pagination_div is not None:
        pages = pagination_div.find_all('a')
        total_pages = int(pages[-2].text.strip())
    else:
        total_pages = 1
        
    # Extract the desired columns
    data = []
    
    print(total_pages)
    # Iterate over the pages
    for page in range(1, total_pages + 1):
        problem_page_url = f"{problem_url}{page}?friends=on"
        print(problem_page_url)

        # Send a GET request to the URL
        response = session.get(problem_page_url)
        
        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find('table', {'class': 'status-frame-datatable'})
        rows = table.find_all('tr')

        for row in rows[1:]:
            cells = row.find_all('td')
            if len(cells) >= 8:
                num = cells[0].text.strip()
                when = cells[1].text.strip()
                who = cells[2].text.strip()
                problem = cells[3].text.strip()
                lang = cells[4].text.strip()
                verdict = cells[5].text.strip()
                time = cells[6].text.strip()
                memory = cells[7].text.strip()

                data.append([num, when, who, problem, lang, verdict, time, memory])

    # Create a DataFrame from the extracted data
    friends_problem_df = pd.DataFrame(data, columns=header)
    friends_problemset_df = pd.concat([friends_problemset_df,friends_problem_df], axis=0)


# Iterate over the values in 'Number' and 'Character' columns
for num, char in zip(problemset_df['Number'], problemset_df['Class']):
    
    # Access the protected page
    page = 1
    problem_url = f"{base_url}{num}/problem/{char}/page/"
    problem_page_url = f"{problem_url}{page}?friends=on"
    response = session.get(problem_page_url)
    soup = BeautifulSoup(response.content, 'lxml')
    s = soup.find('div', id= 'body')
    
    # Extract the total number of pages
    pagination_div = s.find('div', style='margin-top:1em;' ).find('div',class_= 'pagination')
    if pagination_div is not None:
        pages = pagination_div.find_all('a')
        total_pages = int(pages[-2].text.strip())
    else:
        total_pages = 1
        
    # Extract the desired columns
    data = []
    
    print(total_pages)
    # Iterate over the pages
    for page in range(1, total_pages + 1):
        problem_page_url = f"{problem_url}{page}?friends=on"
        print(problem_page_url)

        # Send a GET request to the URL
        response = session.get(problem_page_url)
        
        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find('table', {'class': 'status-frame-datatable'})
        rows = table.find_all('tr')

        for row in rows[1:]:
            cells = row.find_all('td')
            if len(cells) >= 8:
                num = cells[0].text.strip()
                when = cells[1].text.strip()
                who = cells[2].text.strip()
                problem = cells[3].text.strip()
                lang = cells[4].text.strip()
                verdict = cells[5].text.strip()
                time = cells[6].text.strip()
                memory = cells[7].text.strip()

                data.append([num, when, who, problem, lang, verdict, time, memory])

    # Create a DataFrame from the extracted data
    friends_problem_df = pd.DataFrame(data, columns=header)
    friends_problemset_df = pd.concat([rama,friends_problem_df], axis=0)

3
https://codeforces.com/problemset/status/4/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/4/problem/A/page/2?friends=on
https://codeforces.com/problemset/status/4/problem/A/page/3?friends=on
3
https://codeforces.com/problemset/status/71/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/71/problem/A/page/2?friends=on
https://codeforces.com/problemset/status/71/problem/A/page/3?friends=on
3
https://codeforces.com/problemset/status/231/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/231/problem/A/page/2?friends=on
https://codeforces.com/problemset/status/231/problem/A/page/3?friends=on
2
https://codeforces.com/problemset/status/1/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/1/problem/A/page/2?friends=on
3
https://codeforces.com/problemset/status/158/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/158/problem/A/page/2?friends=on
https://codeforces.com/problemset/status/158/problem/A

2
https://codeforces.com/problemset/status/443/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/443/problem/A/page/2?friends=on
1
https://codeforces.com/problemset/status/131/problem/A/page/1?friends=on
2
https://codeforces.com/problemset/status/996/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/996/problem/A/page/2?friends=on
2
https://codeforces.com/problemset/status/25/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/25/problem/A/page/2?friends=on
2
https://codeforces.com/problemset/status/268/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/268/problem/A/page/2?friends=on
2
https://codeforces.com/problemset/status/785/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/785/problem/A/page/2?friends=on
2
https://codeforces.com/problemset/status/337/problem/A/page/1?friends=on
https://codeforces.com/problemset/status/337/problem/A/page/2?friends=on
2
https://codeforces.com/problemset/sta

In [8]:
# friends_problemset_df.to_csv('friends_problemset_df_Rama.csv',index=False, encoding = 'utf-8')
friends_problemset_df = pd.read_csv("friends_problemset_df.csv")

### Preprocessing Friends-Problemset Dataset

###### First,
1. Preprocessing Problem column
   1) Splitting Problem column to Number_Class column and Problem_name column
   2) Splitting Number_Class column to Number column and Class column

In [9]:
friends_problemset_df[['Number_Class','Problem_name']] = friends_problemset_df['Problem'].str.split(' - ', expand=True)

# Split the values in the 'Number_character' column
friends_problemset_df['Number'] = friends_problemset_df['Number_Class'].str[:-1].astype(str)
friends_problemset_df['Class'] = friends_problemset_df['Number_Class'].str[-1].astype(str)


friends_problemset_df = friends_problemset_df.drop(columns = ['Problem'])
friends_problemset_df

,#,When,Who,Language,Time,Memory,Number_Class,Problem_name,Number,Class
0,164909440,Jul/19/2022 17:36,kinannotfound,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A
1,168664652,Aug/17/2022 10:44,kinannotfound,GNU C++20 (64),30 ms,0 KB,4A,Watermelon,4,A
2,147200215,Feb/21/2022 20:13,obadaa.masri,GNU C++14,30 ms,0 KB,4A,Watermelon,4,A
3,172236866,Sep/15/2022 14:51,ShathaS2003,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A
4,37352281,Apr/15/2018 14:50,samer_orfali,GNU C++,30 ms,3200 KB,4A,Watermelon,4,A
...,...,...,...,...,...,...,...,...,...,...
6539,160525370,Jun/14/2022 17:37,DARK-N,GNU C++20 (64),15 ms,0 KB,1692A,Marathon,1692,A
6540,160522552,Jun/14/2022 17:36,_SADIEM_,GNU C++20 (64),15 ms,0 KB,1692A,Marathon,1692,A
6541,160546538,Jun/14/2022 17:46,Diab,GNU C++17,15 ms,0 KB,1692A,Marathon,1692,A
6542,201770743,Apr/12/2023 05:18,obada_katma,GNU C++20 (64),15 ms,100 KB,1692A,Marathon,1692,A


In [10]:
##########################come back here to save the work
friends_problemset_df.to_csv('friends_problemset_df_without_source_code.csv', index=False, encoding = 'utf-8')
# friends_problemset_df = pd.read_csv('friends_problemset_df_without_source_code.csv')



##### Second
2. Preprocessing Who column
###### I have only 121 friends but here the unique values of my friends is more than 121

In [224]:
print(friends_problemset_df['Who'].unique(),len(friends_problemset_df['Who'].unique()))

['kinannotfound' 'obadaa.masri' 'ShathaS2003' 'samer_orfali'
 'Mohammad_Kartoumeh' 'yaser.harba' '_SADIEM_' 'Karam_mohmd' 'amarashujaa'
 '099450735z' 'IT-24-' '7amzahsa' 'Abd_HM' '_OmarAbdo' 'Alaa-Kaadan'
 'masa_beydoun' 'codeiologist' 'Grapeee' '3bdo_Farah' 'Bicso'
 'mohammed_kurdi' 'RetiringNextCentury' 'M.Qattan' 'HASSAN_GH'
 'Bassam1412' 'Nour-Aldeen' '_s_h_a_m__' 'abbas.az408' 'AbodeKu'
 'Mohammed--alhefawe' 'Wiaam_SA' 'sarah.m.akkad' 'Ansam092' 'Conan_y65'
 'ali.alnahhas.22' '7nooOoOoon' 'sidratello9' 'Diab' 'ghaith.clash88'
 'NawarAlTibi' 'Brre.Hemo' 'Qddo' 'anas_allaham' 'sanji333333' 'Kareembo'
 'A.mohamad' 'Abdulrhaim_Ma' 'Tashleef' 'RamaRehawi' 'abdalrhman.alkafry'
 'Zaid_Al_Habbal' 'ghina.alrefai' 'MarioAndrawos' 'sepsa.m' 'Sarah_nn'
 'Hamza_99' 'Ahmed-yaser' 'OmranMk' 'AbdAR' 'Baraa_Armoush' 'AhmadShahal'
 'a7mad_taieb' 'Zeina_Albarre' 'Anti_tourist4000' '_Salty'
 'The_soviet_cat' 'Zaher' 'ZENGOO' 'Mounir_Maleh' 'abod3e9' 'Raed_Sb'
 'Lanakhaled' 'Amr_Kart' 'Bsher' 'DARK-N'

In [225]:
# Replace rows with '#' at the end of 'Who' values with the value without '#'
friends_problemset_df['Who'] = friends_problemset_df['Who'].apply(lambda x: str(x).replace('#', ''))
# friends_problemset_df = friends_problemset_df[~friends_problemset_df['Who'].str.endswith('#')]

# Replace rows with '* ' at the beginning of 'Who' values with the value without '* '
friends_problemset_df['Who'] = friends_problemset_df['Who'].apply(lambda x: str(x).replace('* ', ''))
# friends_problemset_df = friends_problemset_df[~friends_problemset_df['Who'].str.startswith('*')]

print(friends_problemset_df['Who'].unique(), len(friends_problemset_df['Who'].unique()))

['kinannotfound' 'obadaa.masri' 'ShathaS2003' 'samer_orfali'
 'Mohammad_Kartoumeh' 'yaser.harba' '_SADIEM_' 'Karam_mohmd' 'amarashujaa'
 '099450735z' 'IT-24-' '7amzahsa' 'Abd_HM' '_OmarAbdo' 'Alaa-Kaadan'
 'masa_beydoun' 'codeiologist' 'Grapeee' '3bdo_Farah' 'Bicso'
 'mohammed_kurdi' 'RetiringNextCentury' 'M.Qattan' 'HASSAN_GH'
 'Bassam1412' 'Nour-Aldeen' '_s_h_a_m__' 'abbas.az408' 'AbodeKu'
 'Mohammed--alhefawe' 'Wiaam_SA' 'sarah.m.akkad' 'Ansam092' 'Conan_y65'
 'ali.alnahhas.22' '7nooOoOoon' 'sidratello9' 'Diab' 'ghaith.clash88'
 'NawarAlTibi' 'Brre.Hemo' 'Qddo' 'anas_allaham' 'sanji333333' 'Kareembo'
 'A.mohamad' 'Abdulrhaim_Ma' 'Tashleef' 'RamaRehawi' 'abdalrhman.alkafry'
 'Zaid_Al_Habbal' 'ghina.alrefai' 'MarioAndrawos' 'sepsa.m' 'Sarah_nn'
 'Hamza_99' 'Ahmed-yaser' 'OmranMk' 'AbdAR' 'Baraa_Armoush' 'AhmadShahal'
 'a7mad_taieb' 'Zeina_Albarre' 'Anti_tourist4000' '_Salty'
 'The_soviet_cat' 'Zaher' 'ZENGOO' 'Mounir_Maleh' 'abod3e9' 'Raed_Sb'
 'Lanakhaled' 'Amr_Kart' 'Bsher' 'DARK-N'

##### Third, 
3. Preprocessing data to keep only the C++ and Accepted submissions

In [11]:
# Keep only rows with "Accepted" verdict
friends_problemset_df = friends_problemset_df[friends_problemset_df['Verdict'].str.contains("Accepted", regex=False)]

friends_problemset_df = friends_problemset_df.drop(columns = ['Verdict'])

# Keep only rows with "C++" language
friends_problemset_df = friends_problemset_df[friends_problemset_df['Language'].str.contains("C++", regex=False)]

friends_problemset_df

,#,When,Who,Language,Time,Memory,Number_Class,Problem_name,Number,Class
0,164909440,Jul/19/2022 17:36,kinannotfound,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A
1,168664652,Aug/17/2022 10:44,kinannotfound,GNU C++20 (64),30 ms,0 KB,4A,Watermelon,4,A
2,147200215,Feb/21/2022 20:13,obadaa.masri,GNU C++14,30 ms,0 KB,4A,Watermelon,4,A
3,172236866,Sep/15/2022 14:51,ShathaS2003,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A
4,37352281,Apr/15/2018 14:50,samer_orfali,GNU C++,30 ms,3200 KB,4A,Watermelon,4,A
...,...,...,...,...,...,...,...,...,...,...
6539,160525370,Jun/14/2022 17:37,DARK-N,GNU C++20 (64),15 ms,0 KB,1692A,Marathon,1692,A
6540,160522552,Jun/14/2022 17:36,_SADIEM_,GNU C++20 (64),15 ms,0 KB,1692A,Marathon,1692,A
6541,160546538,Jun/14/2022 17:46,Diab,GNU C++17,15 ms,0 KB,1692A,Marathon,1692,A
6542,201770743,Apr/12/2023 05:18,obada_katma,GNU C++20 (64),15 ms,100 KB,1692A,Marathon,1692,A


###### Fourth,
4. Preprocessing When column

In [12]:
friends_problemset_df['When'] = pd.to_datetime(friends_problemset_df['When'], errors = 'coerce')
friends_problemset_df['When'] = friends_problemset_df['When'].dt.strftime('%Y/%m/%d %H:%M')
friends_problemset_df.sort_values("When").head()

,#,When,Who,Language,Time,Memory,Number_Class,Problem_name,Number,Class
4103,3497724,2013/04/09 22:51,RedNextCentury,GNU C++,15 ms,0 KB,131A,cAPS lOCK,131,A
135,3499219,2013/04/10 15:14,RedNextCentury,GNU C++,15 ms,100 KB,71A,Way Too Long Words,71,A
451,3499268,2013/04/10 15:31,RedNextCentury,GNU C++,15 ms,0 KB,158A,Next Round,158,A
102,3499298,2013/04/10 15:38,RedNextCentury,GNU C++,15 ms,0 KB,4A,Watermelon,4,A
1240,3499362,2013/04/10 15:53,RedNextCentury,GNU C++,15 ms,0 KB,118A,String Task,118,A


In [13]:
############come back here to save the work
friends_problemset_df.to_csv('Preprocessed_friends_problemset_df_without_source_code.csv', index=False)

###### Fifth,
5. Friends submission on first 50 problems in problemset

In [17]:
friends_problemset_df['Number']=friends_problemset_df['Number'].astype(str)

problemset_50_df['Number']=problemset_50_df['Number'].astype(str)

friends_problemset_50_df = friends_problemset_df.merge(problemset_50_df, on = ['Number', 'Class', 'Number_Class'])

friends_problemset_50_df


C:\Users\dralh\AppData\Local\Temp\ipykernel_10088\1327453393.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problemset_50_df['Number']=problemset_50_df['Number'].astype(str)


,#,When,Who,Language,Time,Memory,Number_Class,Problem_name,Number,Class,...,Tag_greedy,Tag_hashing,Tag_implementation,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers
0,164909440,2022/07/19 17:36,kinannotfound,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A,...,0,0,0,1,0,0,0,0,0,0
1,168664652,2022/08/17 10:44,kinannotfound,GNU C++20 (64),30 ms,0 KB,4A,Watermelon,4,A,...,0,0,0,1,0,0,0,0,0,0
2,147200215,2022/02/21 20:13,obadaa.masri,GNU C++14,30 ms,0 KB,4A,Watermelon,4,A,...,0,0,0,1,0,0,0,0,0,0
3,172236866,2022/09/15 14:51,ShathaS2003,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A,...,0,0,0,1,0,0,0,0,0,0
4,37352281,2018/04/15 14:50,samer_orfali,GNU C++,30 ms,3200 KB,4A,Watermelon,4,A,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3874,134351371,2021/11/05 01:59,Ahmed-yaser,GNU C++17,31 ms,1400 KB,580A,Kefa and First Steps,580,A,...,0,0,1,0,0,0,0,0,0,0
3875,165058924,2022/07/21 01:16,Mohamed-Omran,GNU C++20 (64),31 ms,400 KB,580A,Kefa and First Steps,580,A,...,0,0,1,0,0,0,0,0,0,0
3876,72092419,2020/02/29 13:59,MouazBudier,GNU C++17,46 ms,400 KB,580A,Kefa and First Steps,580,A,...,0,0,1,0,0,0,0,0,0,0
3877,115429295,2021/05/06 17:53,DARK-N,GNU C++17,46 ms,4200 KB,580A,Kefa and First Steps,580,A,...,0,0,1,0,0,0,0,0,0,0


## Features engineering

###### First,
2. Feature's name: A, B, C, ....
   Splitting Class into multiple Classes columns that has either 0 or 1

In [30]:
def assign_values_based_on_column(df, column):
    unique_values = df[column].unique().tolist()
    df = df.assign(**{k: 0 for k in unique_values})

    for value in unique_values:
        df[value] = (df[column] == value).astype(int)

    return df

In [32]:
friends_problemset_50_df = assign_values_based_on_column(friends_problemset_50_df, 'Class')
friends_problemset_50_df

,#,When,Who,Language,Time,Memory,Number_Class,Problem_name,Number,Class,...,Tag_implementation,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B
0,164909440,2022/07/19 17:36,kinannotfound,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A,...,0,1,0,0,0,0,0,0,1,0
1,168664652,2022/08/17 10:44,kinannotfound,GNU C++20 (64),30 ms,0 KB,4A,Watermelon,4,A,...,0,1,0,0,0,0,0,0,1,0
2,147200215,2022/02/21 20:13,obadaa.masri,GNU C++14,30 ms,0 KB,4A,Watermelon,4,A,...,0,1,0,0,0,0,0,0,1,0
3,172236866,2022/09/15 14:51,ShathaS2003,GNU C++17,30 ms,0 KB,4A,Watermelon,4,A,...,0,1,0,0,0,0,0,0,1,0
4,37352281,2018/04/15 14:50,samer_orfali,GNU C++,30 ms,3200 KB,4A,Watermelon,4,A,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3874,134351371,2021/11/05 01:59,Ahmed-yaser,GNU C++17,31 ms,1400 KB,580A,Kefa and First Steps,580,A,...,1,0,0,0,0,0,0,0,1,0
3875,165058924,2022/07/21 01:16,Mohamed-Omran,GNU C++20 (64),31 ms,400 KB,580A,Kefa and First Steps,580,A,...,1,0,0,0,0,0,0,0,1,0
3876,72092419,2020/02/29 13:59,MouazBudier,GNU C++17,46 ms,400 KB,580A,Kefa and First Steps,580,A,...,1,0,0,0,0,0,0,0,1,0
3877,115429295,2021/05/06 17:53,DARK-N,GNU C++17,46 ms,4200 KB,580A,Kefa and First Steps,580,A,...,1,0,0,0,0,0,0,0,1,0


In [33]:
#########################come back here to save the work
friends_problemset_50_df.to_csv('Preprocessed_friends_problemset_50_df_without_source_code.csv', index=False, encoding = 'utf-8')
# friends_problemset_50_df = pd.read_csv("Preprocessed_friends_problemset_50_df_without_source_code.csv")

In [34]:
# Split the DataFrame into four equal-sized DataFrames
split_data = np.array_split(friends_problemset_50_df, 4)

# Access the individual DataFrames
friends_problemset_50_df1, friends_problemset_50_df2, friends_problemset_50_df3, friends_problemset_50_df4 = split_data[0], split_data[1], split_data[2], split_data[3]

# Check the sizes of the DataFrames
print(len(friends_problemset_50_df1)) 
print(len(friends_problemset_50_df2)) 
print(len(friends_problemset_50_df3))  
print(len(friends_problemset_50_df4))  

970
970
970
969


##### We will now scrape the source code for each of the friends_problemset_50_df1,  riends_problemset_50_df2, friends_problemset_50_df3, friends_problemset_50_df4 but on different laptops (so that we can get more requests)

In [120]:
if os.path.exists('failed_submission.csv'):
    failed_submission = pd.read_csv("failed_submission.csv")
    
    # Find the index where starting_point['#'] is equal to the specific value
    start_index = friends_problemset_50_df1[friends_problemset_50_df1['#'] == failed_submission['#'][0]].index[0]
    
    # Slice the DataFrame to include rows starting from the identified index
    starting_point = friends_problemset_50_df1.loc[start_index:]
    os.remove('failed_submission.csv')

else:
    # Slice the DataFrame to include rows starting from the identified index
    starting_point = friends_problemset_50_df1
    

# Iterate over the values in 'Number' and '#' columns
for num, submission in zip(starting_point['Number'], starting_point['#']):
        # The base URL for the submission page
        submission_url = f'https://codeforces.com/contest/{num}/submission/{submission}'
        print(submission_url)

        # Send a GET request to the URL
        response = requests.get(submission_url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the element containing the source code
            source_elem = soup.select('#program-source-text')

            # Check if the element was found
            if source_elem:
                # Convert the HTML source code to plain text
                source = html.unescape(source_elem[0].decode_contents())

                # Path to the successful_submission file
                successful_submissions = "successful_submissions.csv"

                # Open the file in append mode and write the data
                with open(successful_submissions, "a", newline="", encoding='utf-8') as file:
                    writer = csv.DictWriter(file, fieldnames=["#", "Number", 'Source_code'])

                    # Write the header row if the file is empty
                    if file.tell() == 0:
                        writer.writeheader()

                    # Append the data rows
                    writer.writerows([{"#": submission, "Number": num, "Source_code":source}])
            else:
                print(f'Failed to find source code for submission {submission} in contest {num}')
                                                
                # Path to the failed_submission.csv file
                failed_submission = "failed_submission.csv"

                # Open the file in append mode and write the data
                with open(failed_submission, "w", newline="", encoding='utf-8') as file:
                    writer = csv.DictWriter(file, fieldnames=["#", "Number"])

                    # Write the header row if the file is empty
                    if file.tell() == 0:
                        writer.writeheader()

                    # Append the data rows
                    writer.writerows([{"#": submission, "Number": num}])
                    
                winsound.PlaySound('frog.wav', winsound.SND_FILENAME)
                break
                
        else:
            print(f'Failed to retrieve submission {submission} in contest {num}')
        time.sleep(5)
        

https://codeforces.com/contest/339/submission/196939273
https://codeforces.com/contest/339/submission/170392948
https://codeforces.com/contest/339/submission/202082934
https://codeforces.com/contest/339/submission/71821352
https://codeforces.com/contest/339/submission/158241038
https://codeforces.com/contest/339/submission/175678154
https://codeforces.com/contest/339/submission/58838519
https://codeforces.com/contest/339/submission/192504043
https://codeforces.com/contest/339/submission/166609153
https://codeforces.com/contest/339/submission/169744556
https://codeforces.com/contest/339/submission/160079851
https://codeforces.com/contest/339/submission/161365443
https://codeforces.com/contest/339/submission/159533920
https://codeforces.com/contest/339/submission/79610758
https://codeforces.com/contest/339/submission/196006462
https://codeforces.com/contest/339/submission/101964930
https://codeforces.com/contest/339/submission/71916954
https://codeforces.com/contest/339/submission/109219

##### After repeating the same code for the 4 dataframes and concatinating them 

In [35]:
Problem_submissions_source_code_50_df = pd.read_csv('all_successful_submissions.csv')
Problem_submissions_source_code_50_df['Number'] = Problem_submissions_source_code_50_df['Number'].astype(str)

In [36]:
friends_submissions_source_code_problemset_50_df = Problem_submissions_source_code_50_df.merge(friends_problemset_50_df, on = ['#', 'Number'], how = 'left')
friends_submissions_source_code_problemset_50_df

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Problem_name,...,Tag_implementation,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B
0,164909440,4,#include <iostream>\r\nint main()\r\n{\r\n in...,2022/07/19 17:36,kinannotfound,GNU C++17,30 ms,0 KB,4A,Watermelon,...,0,1,0,0,0,0,0,0,1,0
1,168664652,4,#include <iostream>\r\n using namespace std...,2022/08/17 10:44,kinannotfound,GNU C++20 (64),30 ms,0 KB,4A,Watermelon,...,0,1,0,0,0,0,0,0,1,0
2,147200215,4,#include<iostream>\r\nusing namespace std;\r\n...,2022/02/21 20:13,obadaa.masri,GNU C++14,30 ms,0 KB,4A,Watermelon,...,0,1,0,0,0,0,0,0,1,0
3,172236866,4,#include<iostream>\r\nusing namespace std;\r\n...,2022/09/15 14:51,ShathaS2003,GNU C++17,30 ms,0 KB,4A,Watermelon,...,0,1,0,0,0,0,0,0,1,0
4,37352281,4,#include<stdio.h>\r\nint main()\r\n{\r\n in...,2018/04/15 14:50,samer_orfali,GNU C++,30 ms,3200 KB,4A,Watermelon,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3874,172090257,520,#include <bits/stdc++.h>\r\n\r\nusing namespac...,2022/09/14 02:44,M.Qattan,GNU C++17,15 ms,100 KB,520A,Pangram,...,1,0,0,0,0,0,1,0,1,0
3875,192212595,520,#include<bits/stdc++.h>\r\n\r\nusing namespace...,2023/02/05 01:12,7amzahsa,GNU C++20 (64),15 ms,0 KB,520A,Pangram,...,1,0,0,0,0,0,1,0,1,0
3876,36421933,580,#include <iostream>\r\n\r\nusing namespace std...,2018/03/20 02:10,samer_orfali,GNU C++,264 ms,4200 KB,580A,Kefa and First Steps,...,1,0,0,0,0,0,0,0,1,0
3877,162374843,580,#include <bits/stdc++.h>\r\n\r\nusing namespac...,2022/06/30 22:19,a7mad_taieb,GNU C++20 (64),218 ms,400 KB,580A,Kefa and First Steps,...,1,0,0,0,0,0,0,0,1,0


In [37]:
#########################Come back here to save the work
friends_submissions_source_code_problemset_50_df.to_csv('friends_submissions_source_code_problemset_50.csv', index = False, encoding = 'utf-8')
# friends_submissions_source_code_problemset_50_df = pd.read_csv("friends_submissions_source_code_problemset_50.csv")

In [38]:
friends_submissions_source_code_problemset_50_df.columns

Index(['#', 'Number', 'Source_code', 'When', 'Who', 'Language', 'Time',
       'Memory', 'Number_Class', 'Problem_name', 'Class', 'Difficulty',
       'Solved', 'Problem Name', 'Tag_binarysearch', 'Tag_bitmasks',
       'Tag_bruteforce', 'Tag_constructivealgorithms', 'Tag_datastructures',
       'Tag_dp', 'Tag_expressionparsing', 'Tag_graphmatchings', 'Tag_greedy',
       'Tag_hashing', 'Tag_implementation', 'Tag_math', 'Tag_numbertheory',
       'Tag_shortestpaths', 'Tag_sortings', 'Tag_specialproblem',
       'Tag_strings', 'Tag_twopointers', 'A', 'B'],
      dtype='object')

### Preprocessing Friends-Problemset Dataset after scraping the source_codes

###### Fifth,
5. Preprocessing Problem_name and Problem Name columns

In [39]:
friends_submissions_source_code_problemset_50_df.drop(columns=['Problem_name'], inplace=True)


friends_submissions_source_code_problemset_50_df['Problem Name'] = friends_submissions_source_code_problemset_50_df['Problem Name'].str.replace(" ", "")


friends_submissions_source_code_problemset_50_df['Problem Name'] = friends_submissions_source_code_problemset_50_df['Problem Name'].str.replace("-", "")


friends_submissions_source_code_problemset_50_df['Problem Name'] = friends_submissions_source_code_problemset_50_df['Problem Name'].str.replace(" ", "")


friends_submissions_source_code_problemset_50_df['Problem Name'].unique()

array(['Watermelon', 'WayTooLongWords', 'Team', 'TheatreSquare',
       'NextRound', 'Dominopiling', 'Bit++', 'BeautifulMatrix',
       'PetyaandStrings', 'HelpfulMaths', 'WordCapitalization',
       'BoyorGirl', 'StringTask', 'StonesontheTable', 'BearandBigBrother',
       'SoldierandBananas', 'Elephant', 'Word', 'YoungPhysicist',
       'WrongSubtraction', 'Football', 'NearlyLuckyNumber',
       'AntonandDanik', 'Tram', 'Translation', 'QueueattheSchool',
       'VanyaandFence', 'BeautifulYear', 'Chatroom', 'LuckyDivision',
       'InSearchofanEasyProblem', 'Twins', 'GeorgeandAccommodation',
       'Magnets', 'Presents', 'CalculatingFunction', 'Drinks', 'EvenOdds',
       'UltraFastMathematician', 'HQ9+', 'Hulk',
       'Isyourhorseshoeontheotherhoof?', 'GravityFlip', 'IWannaBetheGuy',
       'Expression', 'DivisibilityProblem', 'ArrivaloftheGeneral',
       'Insomniacure', 'Pangram', 'KefaandFirstSteps'], dtype=object)

###### Seventh,
7. One submission per a student per a problem
 
 ##### We don't need multiple problems submissions by the student, so we will keep only the oldest submission in date

In [40]:
def one_submission_per_student_per_problem(df):

    sorted_df = df.sort_values('When')  # Sort DataFrame by 'When' column in ascending order

    mask = sorted_df.duplicated(['Who','Number_Class','Number','Class','Problem Name'], keep='first')  # Create mask to keep the older row for each unique ['Who','Number_Class'] values

    return sorted_df[~mask]

In [41]:
preprocessed_friends_submissions_source_code_problemset_50_df = one_submission_per_student_per_problem(friends_submissions_source_code_problemset_50_df)
preprocessed_friends_submissions_source_code_problemset_50_df

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_implementation,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B
135,3499219,71,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:14,RedNextCentury,GNU C++,15 ms,100 KB,71A,A,...,0,0,0,0,0,0,1,0,1,0
451,3499268,158,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 15:31,RedNextCentury,GNU C++,15 ms,0 KB,158A,A,...,1,0,0,0,0,1,0,0,1,0
102,3499298,4,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:38,RedNextCentury,GNU C++,15 ms,0 KB,4A,A,...,0,1,0,0,0,0,0,0,1,0
1240,3499362,118,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:53,RedNextCentury,GNU C++,15 ms,0 KB,118A,A,...,1,0,0,0,0,0,1,0,1,0
2092,3499574,116,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 16:49,RedNextCentury,GNU C++,15 ms,0 KB,116A,A,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,207975113,116,#include <iostream>\r\n\r\nusing namespace std...,2023/05/31 17:50,mlakdyab578,GNU C++17,30 ms,0 KB,116A,A,...,1,0,0,0,0,0,0,0,1,0
2326,207995332,677,#include <iostream>\r\n\r\nusing namespace std...,2023/05/31 22:01,mlakdyab578,GNU C++17,15 ms,0 KB,677A,A,...,1,0,0,0,0,0,0,0,1,0
2540,208963348,122,#include<bits/stdc++.h>\r\nusing namespace std...,2023/06/08 10:39,Alaatnt,GNU C++20 (64),30 ms,0 KB,122A,A,...,0,0,1,0,0,0,0,0,1,0
3713,208986965,148,#include <bits/stdc++.h>\r\n#define fast() ...,2023/06/08 14:34,_s_h_a_m__,GNU C++17,30 ms,0 KB,148A,A,...,1,1,0,0,0,0,0,0,1,0


In [42]:
preprocessed_friends_submissions_source_code_problemset_50_df.groupby("Problem Name").count()

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_implementation,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B
Problem Name,,,,,,,,,,,,,,,,,,,,,
AntonandDanik,80,80,80,80,80,80,80,80,80,80,...,80,80,80,80,80,80,80,80,80,80
ArrivaloftheGeneral,48,48,48,48,48,48,48,48,48,48,...,48,48,48,48,48,48,48,48,48,48
BearandBigBrother,77,77,77,77,77,77,77,77,77,77,...,77,77,77,77,77,77,77,77,77,77
BeautifulMatrix,84,84,84,84,84,84,84,84,84,84,...,84,84,84,84,84,84,84,84,84,84
BeautifulYear,57,57,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
Bit++,81,81,81,81,81,81,81,81,81,81,...,81,81,81,81,81,81,81,81,81,81
BoyorGirl,84,84,84,84,84,84,84,84,84,84,...,84,84,84,84,84,84,84,84,84,84
CalculatingFunction,72,72,72,72,72,72,72,72,72,72,...,72,72,72,72,72,72,72,72,72,72
Chatroom,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49


In [43]:
######################################################################come back here to save the work

preprocessed_friends_submissions_source_code_problemset_50_df.to_csv('preprocessed_friends_submissions_source_code_problemset_50_df.csv', index = False, encoding = 'utf-8')


## Preprocessing problemset_50_df Dataset

In [44]:
problemset_50_df['Problem Name'] = problemset_50_df['Problem Name'].str.replace(" ", "")


problemset_50_df['Problem Name'] = problemset_50_df['Problem Name'].str.replace("-", "")


problemset_50_df['Problem Name'].unique()


C:\Users\dralh\AppData\Local\Temp\ipykernel_10088\1876843080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problemset_50_df['Problem Name'] = problemset_50_df['Problem Name'].str.replace(" ", "")
C:\Users\dralh\AppData\Local\Temp\ipykernel_10088\1876843080.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problemset_50_df['Problem Name'] = problemset_50_df['Problem Name'].str.replace("-", "")


array(['Watermelon', 'WayTooLongWords', 'Team', 'TheatreSquare',
       'NextRound', 'Dominopiling', 'Bit++', 'BeautifulMatrix',
       'PetyaandStrings', 'HelpfulMaths', 'WordCapitalization',
       'BoyorGirl', 'StringTask', 'StonesontheTable', 'BearandBigBrother',
       'SoldierandBananas', 'Elephant', 'Word', 'YoungPhysicist',
       'WrongSubtraction', 'Football', 'NearlyLuckyNumber',
       'AntonandDanik', 'Tram', 'Translation', 'QueueattheSchool',
       'VanyaandFence', 'BeautifulYear', 'Chatroom', 'LuckyDivision',
       'InSearchofanEasyProblem', 'Twins', 'GeorgeandAccommodation',
       'Magnets', 'Presents', 'CalculatingFunction', 'Drinks', 'EvenOdds',
       'UltraFastMathematician', 'HQ9+', 'Hulk',
       'Isyourhorseshoeontheotherhoof?', 'GravityFlip', 'IWannaBetheGuy',
       'Expression', 'DivisibilityProblem', 'ArrivaloftheGeneral',
       'Insomniacure', 'Pangram', 'KefaandFirstSteps'], dtype=object)

### Friends-Solved-Problems Dataset

### Before scraping

###### Getting the top friends in solving problems from our problemset
##### we will track only the students who solved at least half the problems in our problemset
##### We will include RamaRehawi also as an exception ;) because she is the greatest in our team memmbers ><

In [45]:
top_friends = preprocessed_friends_submissions_source_code_problemset_50_df.groupby('Who', as_index = False)['#'].count().sort_values('#', ascending= False)
top_friends.rename(columns = {"#": "count"}, inplace =True)
top_72_friends = top_friends.head(72)
top_72_friends

,Who,count
35,DARK-N,50
71,RetiringNextCentury,50
101,codeiologist,50
32,Brre.Hemo,50
41,HASSAN_GH,50
...,...,...
52,Mohamed-Omran,27
21,Alaa-Kaadan,27
40,Grapeee,26
69,Raneem.Alhassan,25


#### Getting all the friends' solved problems that are before their last submission on our problemset 

In [46]:
friends_last_submission_date_on_problemset_50_df = preprocessed_friends_submissions_source_code_problemset_50_df.groupby('Who')['When'].max().reset_index()
friends_last_submission_date_on_problemset_50_df

,Who,When
0,-_Thunder_-,2020/02/04 18:39
1,043_33798_Nuraiym-7,2023/03/04 09:32
2,099450735z,2023/03/04 10:40
3,0xOm4r,2021/12/05 03:30
4,0xessam,2023/01/23 16:45
...,...,...
113,sanji333333,2023/02/21 21:55
114,sarah.m.akkad,2023/02/22 23:53
115,sepsa.m,2020/03/26 21:15
116,sidratello9,2022/11/17 23:50


In [47]:
to_be_scraped = friends_last_submission_date_on_problemset_50_df.merge(top_72_friends, on = 'Who', how = 'inner').sort_values('count', ascending= False)
to_be_scraped

,Who,When,count
35,Mohammed--alhefawe,2022/08/15 19:35,50
41,OmranMk,2023/05/26 19:39,50
45,RetiringNextCentury,2021/06/02 00:52,50
25,HASSAN_GH,2022/06/19 23:18,50
19,Brre.Hemo,2021/03/05 15:43,50
...,...,...,...
12,Alaa-Kaadan,2023/04/23 23:27,27
32,Mohamed-Omran,2022/09/25 22:17,27
24,Grapeee,2023/01/27 17:28,26
43,Raneem.Alhassan,2020/05/11 04:00,25


In [27]:
headers = ['#', 'When', 'Who', 'Problem_name','Lang', 'Number', 'Character']
top_friends_solved_problems_df = pd.DataFrame(columns=headers)

for index, friend in to_be_scraped.iterrows():
  print(friend['Who'])
  # Send GET request to the page
  url = 'https://codeforces.com/submissions/{}'

  friend_name = friend['Who']
  friend['When'] = pd.to_datetime(friend['When'])  
  flag = False

  # Make a GET request to the submission URL with the contest ID and submission ID
  response = requests.get(url.format(friend_name))

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the HTML content using BeautifulSoup
      soup = BeautifulSoup(response.content, 'html.parser')

  pagination_div = soup.find('div', id= 'body').find_all('div',class_= 'pagination')
  if pagination_div is not None and len(pagination_div)>1:
      pages = pagination_div[1].find_all('a')
      total_pages = int(pages[-2].text.strip())

  else:
      total_pages = 1
  print(total_pages)
  # Find the submissions table
  data = []
  for page in range(total_pages, 0, -1):
    page_url = url + '/page/' + str(page)
    print(page_url.format(friend_name))
    if flag == True:
        break
    response = requests.get(page_url.format(friend_name))
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'status-frame-datatable'})
    if table is None:
        print("Error: Submissions table not found")
        exit()
    if response.status_code != 200:
        print("Failed")
        print(page)
        break

    for row in reversed(table.find_all('tr')):
      cols = row.find_all('td')
      if len(cols) > 0:
        if cols[5].text.strip()=='Accepted' and 'C++' in cols[4].text.strip():
            when = cols[1].text.strip()
            if datetime.datetime.strptime(when, '%b/%d/%Y %H:%M') > friend['When']:
                flag =True
                break
            if (cols[0].find('span') and 'hiddenSource' in cols[0].find('span').get('class') ) or\
                    (cols[0].find('a') and 'contest' in cols[0].find('a')['href']):
                num = row.find_all('td', class_="status-small")[1].find('a')['href'].split('/')[2]
                Problem_name = ' '.join(cols[3].text.split('-')[1:]).strip()
                Character = cols[3].text.split('-')[0].strip()
                submission = cols[0].text.strip()
                Language = cols[4].text.strip()
                Verdict = cols[5].text.strip()
                # ['#','When', 'Who', 'Problem_name','Lang' 'Number', 'Character']
                data.append([submission, when, friend_name, Problem_name, Language, num, Character])

  # Create a pandas DataFrame from the rows
  friend_problems_df = pd.DataFrame(data, columns=headers)
  top_friends_solved_problems_df = pd.concat([top_friends_solved_problems_df,friend_problems_df], axis=0)

Mohammed--alhefawe
14
https://codeforces.com/submissions/Mohammed--alhefawe/page/14
https://codeforces.com/submissions/Mohammed--alhefawe/page/13
https://codeforces.com/submissions/Mohammed--alhefawe/page/12
https://codeforces.com/submissions/Mohammed--alhefawe/page/11
OmranMk
23
https://codeforces.com/submissions/OmranMk/page/23
https://codeforces.com/submissions/OmranMk/page/22
https://codeforces.com/submissions/OmranMk/page/21
https://codeforces.com/submissions/OmranMk/page/20
https://codeforces.com/submissions/OmranMk/page/19
https://codeforces.com/submissions/OmranMk/page/18
https://codeforces.com/submissions/OmranMk/page/17
https://codeforces.com/submissions/OmranMk/page/16
https://codeforces.com/submissions/OmranMk/page/15
https://codeforces.com/submissions/OmranMk/page/14
https://codeforces.com/submissions/OmranMk/page/13
https://codeforces.com/submissions/OmranMk/page/12
https://codeforces.com/submissions/OmranMk/page/11
https://codeforces.com/submissions/OmranMk/page/10
https

https://codeforces.com/submissions/Ahmed-yaser/page/40
https://codeforces.com/submissions/Ahmed-yaser/page/39
https://codeforces.com/submissions/Ahmed-yaser/page/38
https://codeforces.com/submissions/Ahmed-yaser/page/37
https://codeforces.com/submissions/Ahmed-yaser/page/36
https://codeforces.com/submissions/Ahmed-yaser/page/35
https://codeforces.com/submissions/Ahmed-yaser/page/34
https://codeforces.com/submissions/Ahmed-yaser/page/33
https://codeforces.com/submissions/Ahmed-yaser/page/32
https://codeforces.com/submissions/Ahmed-yaser/page/31
https://codeforces.com/submissions/Ahmed-yaser/page/30
https://codeforces.com/submissions/Ahmed-yaser/page/29
https://codeforces.com/submissions/Ahmed-yaser/page/28
https://codeforces.com/submissions/Ahmed-yaser/page/27
https://codeforces.com/submissions/Ahmed-yaser/page/26
https://codeforces.com/submissions/Ahmed-yaser/page/25
Diab
34
https://codeforces.com/submissions/Diab/page/34
https://codeforces.com/submissions/Diab/page/33
https://codeforc

https://codeforces.com/submissions/Mohammad_Kartoumeh/page/144
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/143
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/142
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/141
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/140
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/139
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/138
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/137
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/136
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/135
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/134
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/133
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/132
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/131
https://codeforces.com/submissions/Mohammad_Kartoumeh/page/130
https://codeforces.com/submissions/Mohammad_Kartoumeh/p

https://codeforces.com/submissions/dee../page/13
https://codeforces.com/submissions/dee../page/12
https://codeforces.com/submissions/dee../page/11
https://codeforces.com/submissions/dee../page/10
https://codeforces.com/submissions/dee../page/9
https://codeforces.com/submissions/dee../page/8
https://codeforces.com/submissions/dee../page/7
https://codeforces.com/submissions/dee../page/6
https://codeforces.com/submissions/dee../page/5
https://codeforces.com/submissions/dee../page/4
https://codeforces.com/submissions/dee../page/3
_SADIEM_
56
https://codeforces.com/submissions/_SADIEM_/page/56
https://codeforces.com/submissions/_SADIEM_/page/55
https://codeforces.com/submissions/_SADIEM_/page/54
https://codeforces.com/submissions/_SADIEM_/page/53
https://codeforces.com/submissions/_SADIEM_/page/52
https://codeforces.com/submissions/_SADIEM_/page/51
https://codeforces.com/submissions/_SADIEM_/page/50
https://codeforces.com/submissions/_SADIEM_/page/49
https://codeforces.com/submissions/_SADI

https://codeforces.com/submissions/SaeedSabbagh/page/58
https://codeforces.com/submissions/SaeedSabbagh/page/57
https://codeforces.com/submissions/SaeedSabbagh/page/56
https://codeforces.com/submissions/SaeedSabbagh/page/55
https://codeforces.com/submissions/SaeedSabbagh/page/54
https://codeforces.com/submissions/SaeedSabbagh/page/53
https://codeforces.com/submissions/SaeedSabbagh/page/52
https://codeforces.com/submissions/SaeedSabbagh/page/51
https://codeforces.com/submissions/SaeedSabbagh/page/50
https://codeforces.com/submissions/SaeedSabbagh/page/49
https://codeforces.com/submissions/SaeedSabbagh/page/48
https://codeforces.com/submissions/SaeedSabbagh/page/47
https://codeforces.com/submissions/SaeedSabbagh/page/46
https://codeforces.com/submissions/SaeedSabbagh/page/45
https://codeforces.com/submissions/SaeedSabbagh/page/44
_Salty
15
https://codeforces.com/submissions/_Salty/page/15
https://codeforces.com/submissions/_Salty/page/14
https://codeforces.com/submissions/_Salty/page/13


...

https://codeforces.com/submissions/samer_orfali/page/18
https://codeforces.com/submissions/samer_orfali/page/17
https://codeforces.com/submissions/samer_orfali/page/16
https://codeforces.com/submissions/samer_orfali/page/15
https://codeforces.com/submissions/samer_orfali/page/14
https://codeforces.com/submissions/samer_orfali/page/13
https://codeforces.com/submissions/samer_orfali/page/12
https://codeforces.com/submissions/samer_orfali/page/11
https://codeforces.com/submissions/samer_orfali/page/10
https://codeforces.com/submissions/samer_orfali/page/9
https://codeforces.com/submissions/samer_orfali/page/8
sepsa.m
16
https://codeforces.com/submissions/sepsa.m/page/16
https://codeforces.com/submissions/sepsa.m/page/15
https://codeforces.com/submissions/sepsa.m/page/14
https://codeforces.com/submissions/sepsa.m/page/13
https://codeforces.com/submissions/sepsa.m/page/12
https://codeforces.com/submissions/sepsa.m/page/11
https://codeforces.com/submissions/sepsa.m/page/10
https://codeforces

https://codeforces.com/submissions/Conan_y65/page/6
https://codeforces.com/submissions/Conan_y65/page/5
https://codeforces.com/submissions/Conan_y65/page/4
https://codeforces.com/submissions/Conan_y65/page/3
https://codeforces.com/submissions/Conan_y65/page/2
7amzahsa
8
https://codeforces.com/submissions/7amzahsa/page/8
https://codeforces.com/submissions/7amzahsa/page/7
https://codeforces.com/submissions/7amzahsa/page/6
https://codeforces.com/submissions/7amzahsa/page/5
https://codeforces.com/submissions/7amzahsa/page/4
https://codeforces.com/submissions/7amzahsa/page/3
https://codeforces.com/submissions/7amzahsa/page/2
https://codeforces.com/submissions/7amzahsa/page/1
Yaman_Alwaza
92
https://codeforces.com/submissions/Yaman_Alwaza/page/92
https://codeforces.com/submissions/Yaman_Alwaza/page/91
https://codeforces.com/submissions/Yaman_Alwaza/page/90
https://codeforces.com/submissions/Yaman_Alwaza/page/89
https://codeforces.com/submissions/Yaman_Alwaza/page/88
https://codeforces.com/su

https://codeforces.com/submissions/Anti_tourist4000/page/17
https://codeforces.com/submissions/Anti_tourist4000/page/16
https://codeforces.com/submissions/Anti_tourist4000/page/15
https://codeforces.com/submissions/Anti_tourist4000/page/14
https://codeforces.com/submissions/Anti_tourist4000/page/13
https://codeforces.com/submissions/Anti_tourist4000/page/12
https://codeforces.com/submissions/Anti_tourist4000/page/11
https://codeforces.com/submissions/Anti_tourist4000/page/10
https://codeforces.com/submissions/Anti_tourist4000/page/9
https://codeforces.com/submissions/Anti_tourist4000/page/8
https://codeforces.com/submissions/Anti_tourist4000/page/7
https://codeforces.com/submissions/Anti_tourist4000/page/6
https://codeforces.com/submissions/Anti_tourist4000/page/5
Hamza_Alattar.1
5
https://codeforces.com/submissions/Hamza_Alattar.1/page/5
https://codeforces.com/submissions/Hamza_Alattar.1/page/4
https://codeforces.com/submissions/Hamza_Alattar.1/page/3
https://codeforces.com/submission

https://codeforces.com/submissions/abdalkader-s/page/37
https://codeforces.com/submissions/abdalkader-s/page/36
https://codeforces.com/submissions/abdalkader-s/page/35
https://codeforces.com/submissions/abdalkader-s/page/34
https://codeforces.com/submissions/abdalkader-s/page/33
https://codeforces.com/submissions/abdalkader-s/page/32
https://codeforces.com/submissions/abdalkader-s/page/31
https://codeforces.com/submissions/abdalkader-s/page/30
https://codeforces.com/submissions/abdalkader-s/page/29
https://codeforces.com/submissions/abdalkader-s/page/28
https://codeforces.com/submissions/abdalkader-s/page/27
https://codeforces.com/submissions/abdalkader-s/page/26
https://codeforces.com/submissions/abdalkader-s/page/25
https://codeforces.com/submissions/abdalkader-s/page/24
https://codeforces.com/submissions/abdalkader-s/page/23
https://codeforces.com/submissions/abdalkader-s/page/22
https://codeforces.com/submissions/abdalkader-s/page/21
https://codeforces.com/submissions/abdalkader-s/

https://codeforces.com/submissions/Grapeee/page/69
https://codeforces.com/submissions/Grapeee/page/68
https://codeforces.com/submissions/Grapeee/page/67
https://codeforces.com/submissions/Grapeee/page/66
https://codeforces.com/submissions/Grapeee/page/65
https://codeforces.com/submissions/Grapeee/page/64
https://codeforces.com/submissions/Grapeee/page/63
https://codeforces.com/submissions/Grapeee/page/62
https://codeforces.com/submissions/Grapeee/page/61
https://codeforces.com/submissions/Grapeee/page/60
https://codeforces.com/submissions/Grapeee/page/59
https://codeforces.com/submissions/Grapeee/page/58
https://codeforces.com/submissions/Grapeee/page/57
https://codeforces.com/submissions/Grapeee/page/56
https://codeforces.com/submissions/Grapeee/page/55
https://codeforces.com/submissions/Grapeee/page/54
https://codeforces.com/submissions/Grapeee/page/53
https://codeforces.com/submissions/Grapeee/page/52
https://codeforces.com/submissions/Grapeee/page/51
https://codeforces.com/submissi

In [48]:
# top_friends_solved_problems_df.to_csv("top_friends_solved_problems_df_v2.csv", index = False, encoding = 'utf8')
# top_friends_solved_problems_df.to_csv("top_friends_solved_problems_df_v4.csv", index = False, encoding = 'utf8')
# top_friends_solved_problems_df.to_csv("top_friends_solved_problems_df_v4.csv", index = False, encoding = 'utf8')

top_friends_solved_problems_df = pd.read_csv("top_friends_solved_problems_df_v4.csv")
top_friends_solved_problems_df

,#,When,Who,Problem Name,Lang,Number,Class,Number_Class
0,132369900,2021-10-18 19:29:00,Mohammed--alhefawe,Watermelon,GNU C++17 (64),4,A,4A
1,132377185,2021-10-18 21:02:00,Mohammed--alhefawe,WayTooLongWords,GNU C++17 (64),71,A,71A
2,132390336,2021-10-19 01:59:00,Mohammed--alhefawe,Bit++,GNU C++17 (64),282,A,282A
3,157408860,2022-05-16 18:53:00,Mohammed--alhefawe,Team,GNU C++17,231,A,231A
4,157413706,2022-05-16 19:47:00,Mohammed--alhefawe,TheatreSquare,GNU C++17,1,A,1A
...,...,...,...,...,...,...,...,...
31355,79614040,2020-05-10 06:59:00,RamaRehawi,WrongSubtraction,GNU C++17,977,A,977A
31356,79614377,2020-05-10 07:05:00,RamaRehawi,Elephant,GNU C++17,617,A,617A
31357,79617043,2020-05-10 07:49:00,RamaRehawi,NearlyLuckyNumber,GNU C++17,110,A,110A
31358,79617745,2020-05-10 08:03:00,RamaRehawi,Translation,GNU C++17,41,A,41A


In [49]:
top_friends_solved_problems_df.groupby('Who').count()

,#,When,Problem Name,Lang,Number,Class,Number_Class
Who,,,,,,,
043_33798_Nuraiym-7,727,727,727,727,727,727,727
099450735z,34,34,34,34,34,34,34
0xessam,326,326,326,326,326,326,326
3bdo_Farah,1100,1100,1100,1100,1100,1100,1100
7amzahsa,118,118,118,118,118,118,118
...,...,...,...,...,...,...,...
obadaa.masri,335,335,335,335,335,335,335
samer_orfali,951,951,951,951,951,951,951
sanji333333,177,177,177,177,177,177,177


### Preprocessing Friends-Solved-Problems Dataset

In [50]:
top_friends_solved_problems_df.columns

Index(['#', 'When', 'Who', 'Problem Name', 'Lang', 'Number', 'Class',
       'Number_Class'],
      dtype='object')

###### First,
1. Preprocessing Problem_name column

In [51]:
top_friends_solved_problems_df.rename(columns = {"Problem_name": "Problem Name"}, inplace = True)

preprocessed_top_friends_solved_problems_df = top_friends_solved_problems_df

preprocessed_top_friends_solved_problems_df['Problem Name'] = top_friends_solved_problems_df['Problem Name'].str.replace(" ", "")


preprocessed_top_friends_solved_problems_df['Problem Name'] = top_friends_solved_problems_df['Problem Name'].str.replace("-", "")


preprocessed_top_friends_solved_problems_df['Problem Name'].unique()

array(['Watermelon', 'WayTooLongWords', 'Bit++', ..., 'Snails',
       'SecretMessages', "Vasya'sCalendar"], dtype=object)

###### Second,
2. Preprocessing Character and Number cloumns and getting back Character_Class column

In [53]:
preprocessed_top_friends_solved_problems_df.rename(columns={'Character': 'Class'}, inplace=True)
preprocessed_top_friends_solved_problems_df['Number'] = preprocessed_top_friends_solved_problems_df["Number"].astype(str)
preprocessed_top_friends_solved_problems_df['Number_Class'] = preprocessed_top_friends_solved_problems_df['Number'] + preprocessed_top_friends_solved_problems_df['Class']
preprocessed_top_friends_solved_problems_df.head()

,#,When,Who,Problem Name,Lang,Number,Class,Number_Class
0,132369900,2021-10-18 19:29:00,Mohammed--alhefawe,Watermelon,GNU C++17 (64),4,A,4A
1,132377185,2021-10-18 21:02:00,Mohammed--alhefawe,WayTooLongWords,GNU C++17 (64),71,A,71A
2,132390336,2021-10-19 01:59:00,Mohammed--alhefawe,Bit++,GNU C++17 (64),282,A,282A
3,157408860,2022-05-16 18:53:00,Mohammed--alhefawe,Team,GNU C++17,231,A,231A
4,157413706,2022-05-16 19:47:00,Mohammed--alhefawe,TheatreSquare,GNU C++17,1,A,1A


###### Third,
3. Preprocessing When column

In [54]:
preprocessed_top_friends_solved_problems_df['When'] = pd.to_datetime(preprocessed_top_friends_solved_problems_df['When'], errors = 'coerce')
preprocessed_top_friends_solved_problems_df['When'] = preprocessed_top_friends_solved_problems_df['When'].dt.strftime('%Y/%m/%d %H:%M')
preprocessed_top_friends_solved_problems_df.sort_values("When").head()

,#,When,Who,Problem Name,Lang,Number,Class,Number_Class
12329,3497724,2013/04/09 22:51,RedNextCentury,cAPSlOCK,GNU C++,131,A,131A
12330,3499219,2013/04/10 15:14,RedNextCentury,WayTooLongWords,GNU C++,71,A,71A
12331,3499268,2013/04/10 15:31,RedNextCentury,NextRound,GNU C++,158,A,158A
12332,3499298,2013/04/10 15:38,RedNextCentury,Watermelon,GNU C++,4,A,4A
12333,3499362,2013/04/10 15:53,RedNextCentury,StringTask,GNU C++,118,A,118A


### Checking whether top 72 friends submissions on problemset have been scraped

In [55]:
preprocessed_friends_submissions_source_code_problemset_50_df.head()

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_implementation,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B
135,3499219,71,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:14,RedNextCentury,GNU C++,15 ms,100 KB,71A,A,...,0,0,0,0,0,0,1,0,1,0
451,3499268,158,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 15:31,RedNextCentury,GNU C++,15 ms,0 KB,158A,A,...,1,0,0,0,0,1,0,0,1,0
102,3499298,4,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:38,RedNextCentury,GNU C++,15 ms,0 KB,4A,A,...,0,1,0,0,0,0,0,0,1,0
1240,3499362,118,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:53,RedNextCentury,GNU C++,15 ms,0 KB,118A,A,...,1,0,0,0,0,0,1,0,1,0
2092,3499574,116,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 16:49,RedNextCentury,GNU C++,15 ms,0 KB,116A,A,...,1,0,0,0,0,0,0,0,1,0


In [56]:
preprocessed_top_72_friends_submissions_source_code_problemset_50_df = preprocessed_friends_submissions_source_code_problemset_50_df.merge(top_72_friends, on = "Who", how = "inner")
preprocessed_top_72_friends_submissions_source_code_problemset_50_df.head()

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B,count
0,3499219,71,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:14,RedNextCentury,GNU C++,15 ms,100 KB,71A,A,...,0,0,0,0,0,1,0,1,0,44
1,3499268,158,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 15:31,RedNextCentury,GNU C++,15 ms,0 KB,158A,A,...,0,0,0,0,1,0,0,1,0,44
2,3499298,4,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:38,RedNextCentury,GNU C++,15 ms,0 KB,4A,A,...,1,0,0,0,0,0,0,1,0,44
3,3499362,118,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:53,RedNextCentury,GNU C++,15 ms,0 KB,118A,A,...,0,0,0,0,0,1,0,1,0,44
4,3499574,116,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 16:49,RedNextCentury,GNU C++,15 ms,0 KB,116A,A,...,0,0,0,0,0,0,0,1,0,44


In [57]:
t1 = preprocessed_top_72_friends_submissions_source_code_problemset_50_df[['Who', 'Number', 'Class', 'Number_Class', 'Problem Name','When']]
t1['When'] = pd.to_datetime(t1['When'])
t1

C:\Users\dralh\AppData\Local\Temp\ipykernel_10088\4081297740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['When'] = pd.to_datetime(t1['When'])


,Who,Number,Class,Number_Class,Problem Name,When
0,RedNextCentury,71,A,71A,WayTooLongWords,2013-04-10 15:14:00
1,RedNextCentury,158,A,158A,NextRound,2013-04-10 15:31:00
2,RedNextCentury,4,A,4A,Watermelon,2013-04-10 15:38:00
3,RedNextCentury,118,A,118A,StringTask,2013-04-10 15:53:00
4,RedNextCentury,116,A,116A,Tram,2013-04-10 16:49:00
...,...,...,...,...,...,...
2861,abdalrhman.alkafry,61,A,61A,UltraFastMathematician,2023-04-27 19:00:00
2862,abdalrhman.alkafry,479,A,479A,Expression,2023-04-28 18:13:00
2863,abdalrhman.alkafry,263,A,263A,BeautifulMatrix,2023-04-30 17:24:00
2864,abdalrhman.alkafry,266,B,266B,QueueattheSchool,2023-05-02 17:12:00


In [58]:
t2 = preprocessed_top_friends_solved_problems_df[['Who', 'Number', 'Class', 'Number_Class', 'Problem Name','When']]
t2['When'] = pd.to_datetime(t2['When'])
t2

C:\Users\dralh\AppData\Local\Temp\ipykernel_10088\3221790048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['When'] = pd.to_datetime(t2['When'])


,Who,Number,Class,Number_Class,Problem Name,When
0,Mohammed--alhefawe,4,A,4A,Watermelon,2021-10-18 19:29:00
1,Mohammed--alhefawe,71,A,71A,WayTooLongWords,2021-10-18 21:02:00
2,Mohammed--alhefawe,282,A,282A,Bit++,2021-10-19 01:59:00
3,Mohammed--alhefawe,231,A,231A,Team,2022-05-16 18:53:00
4,Mohammed--alhefawe,1,A,1A,TheatreSquare,2022-05-16 19:47:00
...,...,...,...,...,...,...
31355,RamaRehawi,977,A,977A,WrongSubtraction,2020-05-10 06:59:00
31356,RamaRehawi,617,A,617A,Elephant,2020-05-10 07:05:00
31357,RamaRehawi,110,A,110A,NearlyLuckyNumber,2020-05-10 07:49:00
31358,RamaRehawi,41,A,41A,Translation,2020-05-10 08:03:00


In [59]:
def check_problemset_in_top_friends_solved_problems(df1, df2):
    # Check if each row of df1 exists in df2
    included_mask = t1.apply(tuple, axis=1).isin(t2.apply(tuple, axis=1))

    # Select the rows from df1 that are not included in df2
    not_included_rows = t1[~included_mask]

    # Display the not included rows
    return not_included_rows

In [60]:
check_problemset_in_top_friends_solved_problems(t1, t2)

,Who,Number,Class,Number_Class,Problem Name,When


###### Fourth,
4. One submission per student on a problem:
 
 ##### We don't need multiple problems submissions by the student, so we will keep only the oldest submission in date

In [61]:
preprocessed_top_friends_solved_problems_df = one_submission_per_student_per_problem(preprocessed_top_friends_solved_problems_df)
preprocessed_top_friends_solved_problems_df


,#,When,Who,Problem Name,Lang,Number,Class,Number_Class
12329,3497724,2013/04/09 22:51,RedNextCentury,cAPSlOCK,GNU C++,131,A,131A
12330,3499219,2013/04/10 15:14,RedNextCentury,WayTooLongWords,GNU C++,71,A,71A
12331,3499268,2013/04/10 15:31,RedNextCentury,NextRound,GNU C++,158,A,158A
12332,3499298,2013/04/10 15:38,RedNextCentury,Watermelon,GNU C++,4,A,4A
12333,3499362,2013/04/10 15:53,RedNextCentury,StringTask,GNU C++,118,A,118A
...,...,...,...,...,...,...,...,...
514,207380701,2023/05/26 19:20,OmranMk,SortwithStep,GNU C++17,1823,B,1823B
515,207382934,2023/05/26 19:39,OmranMk,Tram,GNU C++17,116,A,116A
16795,207445278,2023/05/27 11:25,_s_h_a_m__,ComparisonString,GNU C++17,1837,B,1837B
16796,207788875,2023/05/29 21:15,_s_h_a_m__,Elections,GNU C++17,1593,A,1593A


In [89]:
t2 = preprocessed_top_friends_solved_problems_df[['Who', 'Number', 'Class', 'Number_Class', 'Problem Name','When']]
t2['When'] = pd.to_datetime(t2['When'])
t2

C:\Users\dralh\AppData\Local\Temp\ipykernel_8012\3221790048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['When'] = pd.to_datetime(t2['When'])


,Who,Number,Class,Number_Class,Problem Name,When
0,RedNextCentury,131,A,131A,cAPSlOCK,2013-04-09 22:51:00
1,RedNextCentury,71,A,71A,WayTooLongWords,2013-04-10 15:14:00
2,RedNextCentury,158,A,158A,NextRound,2013-04-10 15:31:00
3,RedNextCentury,4,A,4A,Watermelon,2013-04-10 15:38:00
4,RedNextCentury,118,A,118A,StringTask,2013-04-10 15:53:00
...,...,...,...,...,...,...
456,OmranMk,1823,B,1823B,SortwithStep,2023-05-26 19:20:00
457,OmranMk,116,A,116A,Tram,2023-05-26 19:39:00
155,_s_h_a_m__,1837,B,1837B,ComparisonString,2023-05-27 11:25:00
156,_s_h_a_m__,1593,A,1593A,Elections,2023-05-29 21:15:00


In [91]:
check_problemset_in_top_friends_solved_problems(t1, t2)

,Who,Number,Class,Number_Class,Problem Name,When


###### Fifth,
5. Dropping Lang column:

In [92]:
preprocessed_top_friends_solved_problems_df = preprocessed_top_friends_solved_problems_df.drop(columns= ['Lang'])
preprocessed_top_friends_solved_problems_df

,#,When,Who,Problem Name,Number,Class,Number_Class
0,3497724,2013/04/09 22:51,RedNextCentury,cAPSlOCK,131,A,131A
1,3499219,2013/04/10 15:14,RedNextCentury,WayTooLongWords,71,A,71A
2,3499268,2013/04/10 15:31,RedNextCentury,NextRound,158,A,158A
3,3499298,2013/04/10 15:38,RedNextCentury,Watermelon,4,A,4A
4,3499362,2013/04/10 15:53,RedNextCentury,StringTask,118,A,118A
...,...,...,...,...,...,...,...
456,207380701,2023/05/26 19:20,OmranMk,SortwithStep,1823,B,1823B
457,207382934,2023/05/26 19:39,OmranMk,Tram,116,A,116A
155,207445278,2023/05/27 11:25,_s_h_a_m__,ComparisonString,1837,B,1837B
156,207788875,2023/05/29 21:15,_s_h_a_m__,Elections,1593,A,1593A


###### Sixth,
6. The top friends solutions as tutorials:
 
 ##### We need someone who has solved all the problems in the problemset to consider his/her solutions as tutorials for the problemset problems

In [62]:
preprocessed_top_friends_solved_problems_df.groupby(['Who']).count().reset_index().sort_values('Number', ascending = False).head(20)

,Who,#,When,Problem Name,Lang,Number,Class,Number_Class
24,Grapeee,3188,3188,3187,3188,3188,3188,3188
34,Mohammad_Kartoumeh,2482,2482,2476,2482,2482,2482,2482
44,RedNextCentury,2447,2447,2439,2447,2447,2447,2447
20,Bsher,1857,1857,1852,1857,1857,1857,1857
50,Yaman_Alwaza,1601,1601,1601,1601,1601,1601,1601
46,SaeedSabbagh,1366,1366,1366,1366,1366,1366,1366
57,abdalkader-s,1346,1346,1342,1346,1346,1346,1346
53,_SADIEM_,1182,1182,1182,1182,1182,1182,1182
68,samer_orfali,913,913,913,913,913,913,913
3,3bdo_Farah,906,906,906,906,906,906,906


We noticed that Mohammad_Kartoumeh and Yaman_Alwaza could be took to be the coaches who we will take their solutions as tutorials, and not as students, but the issue here is that Mohammad_Kartoumeh has only solved 47 out of 50 and Yaman_Alwaza has only solved 37 out of 50, so we will see which problems that Mohammad_Kartoumeh didn't solve and take their solutions from Yaman_Alwaza 

In [63]:
Yaman_Alwaza = preprocessed_top_72_friends_submissions_source_code_problemset_50_df[preprocessed_top_72_friends_submissions_source_code_problemset_50_df['Who'] == 'Yaman_Alwaza']
Mohammad_Kartoumeh = preprocessed_top_72_friends_submissions_source_code_problemset_50_df[preprocessed_top_72_friends_submissions_source_code_problemset_50_df['Who'] == 'Mohammad_Kartoumeh']


result = Yaman_Alwaza[~Yaman_Alwaza['Problem Name'].isin(Mohammad_Kartoumeh['Problem Name'])]


result

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B,count
313,48042711,705,#include <iostream>\r\n\r\nusing namespace std...,2019/01/07 00:18,Yaman_Alwaza,GNU C++17,31 ms,200 KB,705A,A,...,0,0,0,0,0,0,0,1,0,37
327,48097422,405,#include <iostream>\r\n\r\nusing namespace std...,2019/01/08 20:48,Yaman_Alwaza,GNU C++17,30 ms,200 KB,405A,A,...,0,0,0,1,0,0,0,1,0,37
333,48394658,1030,#include <iostream>\r\n\r\nusing namespace std...,2019/01/14 20:03,Yaman_Alwaza,GNU C++17,31 ms,200 KB,1030A,A,...,0,0,0,0,0,0,0,1,0,37


In [64]:
tutorial = pd.concat([Mohammad_Kartoumeh[['Number_Class', 'Source_code']], result[['Number_Class', 'Source_code']]]).reset_index(drop = True)
tutorial.head()

,Number_Class,Source_code
0,1A,#include <iostream>\r\nusing namespace std;\r\...
1,282A,#include <iostream>\r\nusing namespace std;\r\...
2,4A,#include <iostream>\r\nusing namespace std;\r\...
3,158A,#include <iostream>\r\n#include <algorithm>\r\...
4,118A,#include<iostream>\r\n#include<map>\r\n#includ...


In [65]:
preprocessed_top_friends_solved_problems_df = preprocessed_top_friends_solved_problems_df[~preprocessed_top_friends_solved_problems_df['Who'].isin(['Mohammad_Kartoumeh', 'Yaman_Alwaza'])]
preprocessed_top_friends_solved_problems_df

,#,When,Who,Problem Name,Lang,Number,Class,Number_Class
12329,3497724,2013/04/09 22:51,RedNextCentury,cAPSlOCK,GNU C++,131,A,131A
12330,3499219,2013/04/10 15:14,RedNextCentury,WayTooLongWords,GNU C++,71,A,71A
12331,3499268,2013/04/10 15:31,RedNextCentury,NextRound,GNU C++,158,A,158A
12332,3499298,2013/04/10 15:38,RedNextCentury,Watermelon,GNU C++,4,A,4A
12333,3499362,2013/04/10 15:53,RedNextCentury,StringTask,GNU C++,118,A,118A
...,...,...,...,...,...,...,...,...
514,207380701,2023/05/26 19:20,OmranMk,SortwithStep,GNU C++17,1823,B,1823B
515,207382934,2023/05/26 19:39,OmranMk,Tram,GNU C++17,116,A,116A
16795,207445278,2023/05/27 11:25,_s_h_a_m__,ComparisonString,GNU C++17,1837,B,1837B
16796,207788875,2023/05/29 21:15,_s_h_a_m__,Elections,GNU C++17,1593,A,1593A


In [66]:
##################################################come back here to save the file
preprocessed_top_friends_solved_problems_df.to_csv("preprocessed_top_friends_solved_problems_df.csv", encoding = 'utf-8', index = False)
# preprocessed_top_friends_solved_problems_df = pd.read_csv("preprocessed_top_friends_solved_problems_df.csv")

In [ ]:
# top_friends_solved_problems_source_code_50_df = top_friends_solved_problems_source_code_50_df.merge(preprocessed_top_friends_solved_problems_df, on = ['#', 'Number'], how = 'left')
# top_friends_solved_problems_source_code_50_df

## Preprocessing Friends Problemset Dataset

###### We will delete the tutorials also from the friends_problemset dataset 

In [77]:
########################################come bach here to save the work 
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df = preprocessed_top_72_friends_submissions_source_code_problemset_50_df[~preprocessed_top_72_friends_submissions_source_code_problemset_50_df['Who'].isin(['Mohammad_Kartoumeh', 'Yaman_Alwaza'])]
# preprocessed_friends_submissions_source_code_without_tutorials_problemset_50_df.to_csv("preprocessed_friends_submissions_source_code_without_tutorials_problemset_50_df.csv", encoding = 'utf-8', index = False)
# preprocessed_friends_submissions_source_code_without_tutorials_problemset_50_df = pd.read_csv("preprocessed_friends_submissions_source_code_without_tutorials_problemset_50_df.csv")
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B,count
0,3499219,71,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:14,RedNextCentury,GNU C++,15 ms,100 KB,71A,A,...,0,0,0,0,0,1,0,1,0,44
1,3499268,158,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 15:31,RedNextCentury,GNU C++,15 ms,0 KB,158A,A,...,0,0,0,0,1,0,0,1,0,44
2,3499298,4,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:38,RedNextCentury,GNU C++,15 ms,0 KB,4A,A,...,1,0,0,0,0,0,0,1,0,44
3,3499362,118,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:53,RedNextCentury,GNU C++,15 ms,0 KB,118A,A,...,0,0,0,0,0,1,0,1,0,44
4,3499574,116,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 16:49,RedNextCentury,GNU C++,15 ms,0 KB,116A,A,...,0,0,0,0,0,0,0,1,0,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861,203706705,61,#include<iostream>\r\n\r\nusing namespace std;...,2023/04/27 19:00,abdalrhman.alkafry,GNU C++20 (64),15 ms,0 KB,61A,A,...,0,0,0,0,0,0,0,1,0,33
2862,203814475,479,#include <bits/stdc++.h>\r\nusing namespace st...,2023/04/28 18:13,abdalrhman.alkafry,GNU C++20 (64),15 ms,0 KB,479A,A,...,1,0,0,0,0,0,0,1,0,33
2863,204053358,263,#include <bits/stdc++.h>\r\nusing namespace st...,2023/04/30 17:24,abdalrhman.alkafry,GNU C++20 (64),0 ms,0 KB,263A,A,...,0,0,0,0,0,0,0,1,0,33
2864,204262991,266,#include <bits/stdc++.h>\r\nusing namespace st...,2023/05/02 17:12,abdalrhman.alkafry,GNU C++20 (64),30 ms,0 KB,266B,B,...,0,0,1,0,0,0,0,0,1,33


In [78]:
t1 = preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df[['Who', 'Number', 'Class', 'Number_Class', 'Problem Name','When']]
t1

,Who,Number,Class,Number_Class,Problem Name,When
0,RedNextCentury,71,A,71A,WayTooLongWords,2013/04/10 15:14
1,RedNextCentury,158,A,158A,NextRound,2013/04/10 15:31
2,RedNextCentury,4,A,4A,Watermelon,2013/04/10 15:38
3,RedNextCentury,118,A,118A,StringTask,2013/04/10 15:53
4,RedNextCentury,116,A,116A,Tram,2013/04/10 16:49
...,...,...,...,...,...,...
2861,abdalrhman.alkafry,61,A,61A,UltraFastMathematician,2023/04/27 19:00
2862,abdalrhman.alkafry,479,A,479A,Expression,2023/04/28 18:13
2863,abdalrhman.alkafry,263,A,263A,BeautifulMatrix,2023/04/30 17:24
2864,abdalrhman.alkafry,266,B,266B,QueueattheSchool,2023/05/02 17:12


In [79]:
t2 = preprocessed_top_friends_solved_problems_df[['Who', 'Number', 'Class', 'Number_Class', 'Problem Name','When']]
t2

,Who,Number,Class,Number_Class,Problem Name,When
0,Mohammed--alhefawe,4,A,4A,Watermelon,2021/10/18 19:29
1,Mohammed--alhefawe,71,A,71A,WayTooLongWords,2021/10/18 21:02
2,Mohammed--alhefawe,282,A,282A,Bit++,2021/10/19 01:59
3,Mohammed--alhefawe,231,A,231A,Team,2022/05/16 18:53
4,Mohammed--alhefawe,1,A,1A,TheatreSquare,2022/05/16 19:47
...,...,...,...,...,...,...
24473,RamaRehawi,977,A,977A,WrongSubtraction,2020/05/10 06:59
24474,RamaRehawi,617,A,617A,Elephant,2020/05/10 07:05
24475,RamaRehawi,110,A,110A,NearlyLuckyNumber,2020/05/10 07:49
24476,RamaRehawi,41,A,41A,Translation,2020/05/10 08:03


In [80]:
check_problemset_in_top_friends_solved_problems(t1, t2)

,Who,Number,Class,Number_Class,Problem Name,When


### Features Engineering

###### First,
1) Feature's name: Previous_problems_count
##### we want to create a feature for each student that represents the problems number he/she solved before each problem in the problemset

In [81]:
def get_previous_problems_count(df1, df2):
  
    previous_problems_count = {}

    for index, row in df1.iterrows():
        who = row['Who']
        when = row['When']
        count = len(df2[(df2['Who'].astype(str) == str(who)) & (df2['When'] < when)])
        previous_problems_count[index] = count

    previous_problems_count_series = pd.Series(previous_problems_count)
    df1['previous_problems_count'] = previous_problems_count_series
    return df1

In [82]:
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df = get_previous_problems_count(preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df,preprocessed_top_friends_solved_problems_df)
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df


C:\Users\dralh\AppData\Local\Temp\ipykernel_10088\1230353.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['previous_problems_count'] = previous_problems_count_series


,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B,count,previous_problems_count
0,3499219,71,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:14,RedNextCentury,GNU C++,15 ms,100 KB,71A,A,...,0,0,0,0,1,0,1,0,44,1
1,3499268,158,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 15:31,RedNextCentury,GNU C++,15 ms,0 KB,158A,A,...,0,0,0,1,0,0,1,0,44,2
2,3499298,4,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:38,RedNextCentury,GNU C++,15 ms,0 KB,4A,A,...,0,0,0,0,0,0,1,0,44,3
3,3499362,118,# include <string>\r\n# include <iostream>\r\n...,2013/04/10 15:53,RedNextCentury,GNU C++,15 ms,0 KB,118A,A,...,0,0,0,0,1,0,1,0,44,4
4,3499574,116,# include <algorithm>\r\n# include <iostream>\...,2013/04/10 16:49,RedNextCentury,GNU C++,15 ms,0 KB,116A,A,...,0,0,0,0,0,0,1,0,44,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861,203706705,61,#include<iostream>\r\n\r\nusing namespace std;...,2023/04/27 19:00,abdalrhman.alkafry,GNU C++20 (64),15 ms,0 KB,61A,A,...,0,0,0,0,0,0,1,0,33,60
2862,203814475,479,#include <bits/stdc++.h>\r\nusing namespace st...,2023/04/28 18:13,abdalrhman.alkafry,GNU C++20 (64),15 ms,0 KB,479A,A,...,0,0,0,0,0,0,1,0,33,61
2863,204053358,263,#include <bits/stdc++.h>\r\nusing namespace st...,2023/04/30 17:24,abdalrhman.alkafry,GNU C++20 (64),0 ms,0 KB,263A,A,...,0,0,0,0,0,0,1,0,33,62
2864,204262991,266,#include <bits/stdc++.h>\r\nusing namespace st...,2023/05/02 17:12,abdalrhman.alkafry,GNU C++20 (64),30 ms,0 KB,266B,B,...,0,1,0,0,0,0,0,1,33,63


In [83]:
def get_previous_column_count(df, column):
    df_sorted = df.sort_values('When')
    unique_values = df[column].unique()
    for value in unique_values:
        df_sorted[value] = (df_sorted[column] == value).astype(int)
        df_sorted[value] = df_sorted.groupby('Who')[value].cumsum() - df_sorted.groupby('Who')[value].cumsum().groupby(df_sorted['Who']).transform('min')

    df_sorted = df_sorted.sort_index()
    df_sorted.reset_index(drop=True, inplace=True)
    return df_sorted


In [84]:
preprocessed_top_friends_solved_problems_df = get_previous_column_count(preprocessed_top_friends_solved_problems_df, 'Class')
preprocessed_top_friends_solved_problems_df

,#,When,Who,Problem Name,Lang,Number,Class,Number_Class,A,B,...,T,G3,G2,F3,1,S,101,H1,H2,123
0,132369900,2021/10/18 19:29,Mohammed--alhefawe,Watermelon,GNU C++17 (64),4,A,4A,0,0,...,0,0,0,0,0,0,0,0,0,0
1,132377185,2021/10/18 21:02,Mohammed--alhefawe,WayTooLongWords,GNU C++17 (64),71,A,71A,1,0,...,0,0,0,0,0,0,0,0,0,0
2,132390336,2021/10/19 01:59,Mohammed--alhefawe,Bit++,GNU C++17 (64),282,A,282A,2,0,...,0,0,0,0,0,0,0,0,0,0
3,157408860,2022/05/16 18:53,Mohammed--alhefawe,Team,GNU C++17,231,A,231A,3,0,...,0,0,0,0,0,0,0,0,0,0
4,157413706,2022/05/16 19:47,Mohammed--alhefawe,TheatreSquare,GNU C++17,1,A,1A,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24473,79614040,2020/05/10 06:59,RamaRehawi,WrongSubtraction,GNU C++17,977,A,977A,32,24,...,0,0,0,0,0,0,0,0,0,0
24474,79614377,2020/05/10 07:05,RamaRehawi,Elephant,GNU C++17,617,A,617A,33,24,...,0,0,0,0,0,0,0,0,0,0
24475,79617043,2020/05/10 07:49,RamaRehawi,NearlyLuckyNumber,GNU C++17,110,A,110A,34,24,...,0,0,0,0,0,0,0,0,0,0
24476,79617745,2020/05/10 08:03,RamaRehawi,Translation,GNU C++17,41,A,41A,35,24,...,0,0,0,0,0,0,0,0,0,0


In [85]:
preprocessed_top_friends_solved_problems_df.columns

Index(['#', 'When', 'Who', 'Problem Name', 'Lang', 'Number', 'Class',
       'Number_Class', 'A', 'B', 'C', 'D', 'J', 'E', 'L', 'K', 'G', 'H', 'B1',
       'B2', 'G1', 'F', 'I', 'M', 'B3', 'D1', 'D2', 'Y', 'X', 'Z', 'R', 'N',
       'L1', 'L2', 'C1', 'F1', 'E1', 'C2', '100', 'F2', 'E2', 'Q', 'P', 'A1',
       'A2', 'O', 'U', 'W', 'T', 'G3', 'G2', 'F3', '1', 'S', '101', 'H1', 'H2',
       '123'],
      dtype='object')

In [86]:
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df.columns

Index(['#', 'Number', 'Source_code', 'When', 'Who', 'Language', 'Time',
       'Memory', 'Number_Class', 'Class', 'Difficulty', 'Solved',
       'Problem Name', 'Tag_binarysearch', 'Tag_bitmasks', 'Tag_bruteforce',
       'Tag_constructivealgorithms', 'Tag_datastructures', 'Tag_dp',
       'Tag_expressionparsing', 'Tag_graphmatchings', 'Tag_greedy',
       'Tag_hashing', 'Tag_implementation', 'Tag_math', 'Tag_numbertheory',
       'Tag_shortestpaths', 'Tag_sortings', 'Tag_specialproblem',
       'Tag_strings', 'Tag_twopointers', 'A', 'B', 'count',
       'previous_problems_count'],
      dtype='object')

In [87]:
result = preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df[~preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df[['Number','Class','Number_Class','Who', 'Problem Name', 'When']].isin(preprocessed_top_friends_solved_problems_df[['Number','Class','Number_Class','Who', 'Problem Name', 'When']])]
result = result.dropna()
result

,#,Number,Source_code,When,Who,Language,Time,Memory,Number_Class,Class,...,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,A,B,count,previous_problems_count


In [93]:
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df.rename(columns = {"A": "Problem Class A", "B": "Problem Class B"}, inplace = True)
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df.drop(columns = ['#', 'Language','Time','Memory','count'], inplace = True)
preprocessed_top_friends_solved_problems_df.drop(columns = ['#', 'Lang'], inplace = True)

C:\Users\dralh\AppData\Local\Temp\ipykernel_10088\4078741186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df.drop(columns = ['#', 'Language','Time','Memory','count'], inplace = True)


In [94]:
preprocessed_top_friends_solved_problems_df.columns

Index(['When', 'Who', 'Problem Name', 'Number', 'Class', 'Number_Class', 'A',
       'B', 'C', 'D', 'J', 'E', 'L', 'K', 'G', 'H', 'B1', 'B2', 'G1', 'F', 'I',
       'M', 'B3', 'D1', 'D2', 'Y', 'X', 'Z', 'R', 'N', 'L1', 'L2', 'C1', 'F1',
       'E1', 'C2', '100', 'F2', 'E2', 'Q', 'P', 'A1', 'A2', 'O', 'U', 'W', 'T',
       'G3', 'G2', 'F3', '1', 'S', '101', 'H1', 'H2', '123'],
      dtype='object')

In [95]:
preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df.columns

Index(['Number', 'Source_code', 'When', 'Who', 'Number_Class', 'Class',
       'Difficulty', 'Solved', 'Problem Name', 'Tag_binarysearch',
       'Tag_bitmasks', 'Tag_bruteforce', 'Tag_constructivealgorithms',
       'Tag_datastructures', 'Tag_dp', 'Tag_expressionparsing',
       'Tag_graphmatchings', 'Tag_greedy', 'Tag_hashing', 'Tag_implementation',
       'Tag_math', 'Tag_numbertheory', 'Tag_shortestpaths', 'Tag_sortings',
       'Tag_specialproblem', 'Tag_strings', 'Tag_twopointers',
       'Problem Class A', 'Problem Class B', 'previous_problems_count'],
      dtype='object')

In [96]:
students_problemset = preprocessed_top_friends_solved_problems_df.merge(preprocessed_top_72_friends_submissions_source_code_without_tutorials_problemset_50_df, on = ['Number','Class','Number_Class','Who', 'Problem Name', 'When'], how = 'inner')
students_problemset

,When,Who,Problem Name,Number,Class,Number_Class,A,B,C,D,...,Tag_math,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,Problem Class A,Problem Class B,previous_problems_count
0,2021/10/18 19:29,Mohammed--alhefawe,Watermelon,4,A,4A,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,2021/10/18 21:02,Mohammed--alhefawe,WayTooLongWords,71,A,71A,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,2021/10/19 01:59,Mohammed--alhefawe,Bit++,282,A,282A,2,0,0,0,...,0,0,0,0,0,0,0,1,0,2
3,2022/05/16 18:53,Mohammed--alhefawe,Team,231,A,231A,3,0,0,0,...,0,0,0,0,0,0,0,1,0,3
4,2022/05/16 19:47,Mohammed--alhefawe,TheatreSquare,1,A,1A,4,0,0,0,...,1,0,0,0,0,0,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,2020/05/10 06:59,RamaRehawi,WrongSubtraction,977,A,977A,32,24,3,4,...,0,0,0,0,0,0,0,1,0,65
2778,2020/05/10 07:05,RamaRehawi,Elephant,617,A,617A,33,24,3,4,...,1,0,0,0,0,0,0,1,0,66
2779,2020/05/10 07:49,RamaRehawi,NearlyLuckyNumber,110,A,110A,34,24,3,4,...,0,0,0,0,0,0,0,1,0,67
2780,2020/05/10 08:03,RamaRehawi,Translation,41,A,41A,35,24,3,4,...,0,0,0,0,0,1,0,1,0,68


###### Second, Third, Fourth, Fifth
2. Feature's name: tutorial_code_difficulty
3. Features name: tutorial_general_concepts
4. Features name: student_code_difficulty


In [97]:
concepts_level={'الأساسيات': 1,
 'أنواع البيانات': 2,
 'المتغيرات': 2,
 'التعامل مع الأعداد': 3,
 'التعامل مع النصوص': 3,
 'العوامل': 3,
 'المصفوفات': 3,
 'الدوال': 3,
 'الحلقات': 4,
 'الشروط': 4}

required_concepts = {'الأساسيات': [],
                  'أنواع البيانات': ['الأساسيات'],
                  'المتغيرات': ['الأساسيات'],
                  'التعامل مع الأعداد': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'التعامل مع النصوص': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'العوامل': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'المصفوفات': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'الدوال': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'الحلقات': ['العوامل','أنواع البيانات','المتغيرات','الأساسيات'],
                  'الشروط': ['العوامل','أنواع البيانات','المتغيرات','الأساسيات']}

In [98]:
students_problemset.columns

Index(['When', 'Who', 'Problem Name', 'Number', 'Class', 'Number_Class', 'A',
       'B', 'C', 'D', 'J', 'E', 'L', 'K', 'G', 'H', 'B1', 'B2', 'G1', 'F', 'I',
       'M', 'B3', 'D1', 'D2', 'Y', 'X', 'Z', 'R', 'N', 'L1', 'L2', 'C1', 'F1',
       'E1', 'C2', '100', 'F2', 'E2', 'Q', 'P', 'A1', 'A2', 'O', 'U', 'W', 'T',
       'G3', 'G2', 'F3', '1', 'S', '101', 'H1', 'H2', '123', 'Source_code',
       'Difficulty', 'Solved', 'Tag_binarysearch', 'Tag_bitmasks',
       'Tag_bruteforce', 'Tag_constructivealgorithms', 'Tag_datastructures',
       'Tag_dp', 'Tag_expressionparsing', 'Tag_graphmatchings', 'Tag_greedy',
       'Tag_hashing', 'Tag_implementation', 'Tag_math', 'Tag_numbertheory',
       'Tag_shortestpaths', 'Tag_sortings', 'Tag_specialproblem',
       'Tag_strings', 'Tag_twopointers', 'Problem Class A', 'Problem Class B',
       'previous_problems_count'],
      dtype='object')

In [99]:
tutorial.columns

Index(['Number_Class', 'Source_code'], dtype='object')

In [100]:
tutorial = tutorial.assign(**{k: 0 for k in concepts_level.keys()})

In [101]:
keys, values = zip(*concepts_level.items())

In [102]:
def remove_comments(code):
    code = re.sub(r'\/\/.*', '// \\n', code)
    code = re.sub(r'\/\*.*?\*\/', '/* */ \\n', code, flags=re.DOTALL)
    code = code.strip()
    code = re.sub(r'^\s*\n', '', code, flags=re.MULTILINE)

    return code

In [103]:
def count_functions(code):
    pattern = r'\w+\s+\w+\(.*\)\s*{?'
    matches = re.findall(pattern, code)
    return len(matches)

In [105]:
def remove_main_function(code):
    lines = code.split('\n')
    new_lines = []
    is_main_function = False

    for line in lines:
        if 'int main(' in line:
            is_main_function = True
            new_lines.append('\n')
            continue
        elif 'return' in line and is_main_function:
            new_lines.append('\n')
            is_main_function = False
            continue

        new_lines.append(line)

    new_code = '\n'.join(new_lines)
    return new_code

In [106]:
def detect_concept(code):

    # Regular expressions for different declarations
    declaration_patterns = {
        'الحلقات': r'(?:for\s*\([^;]+;[^;]+;[^)]+\))|(?:while\s*\([^)]+\))|(?:do\s*\{[^}]+\}\s*while\s*\([^)]+\))',
        'الشروط': r'(?:if\s*\([^)]+\)(?:\s*\{[^}]*\})*)|else\s*if\s*\([^)]+\)(?:\s*\{[^}]*\})*|else(?:\s*\{[^}]*\})*|switch\s*\([^)]+\)(?:\s*\{[^}]*\})*',
        'المتغيرات': r'(?: (?:float|char|int|double|bool)\s+)+([a-zA-Z_]\w*)\s*(?:\[[^\]]+\])?(?:\s*=\s*(?:[^,;{}()]+|{.*}))?\s*(?:,|\[|\)|;|\{)',
        'أنواع البيانات': r'\b(?:unsigned|signed|const|static|extern|volatile|register|auto|bool|char|short|int|long|float|double|void|string)\b',
        'الأساسيات':r'(cin\s*>>.*|cout\s*<<|//.* ?|\/\*(.|\n)+?\*\/)',
        'التعامل مع الأعداد': r'\b(?:acos|asin|atan|atan2|ceil|cos|cosh|exp|fabs|floor|fmod|frexp|ldexp|log|log10|modf|pow|sin|sinh|sqrt|tan|tanh)\b',
        'التعامل مع النصوص': r'\b(?: string|"?\.(append|substr|length|empty|insert|replace|find))\b',
        'المصفوفات': r'\b\w+\[\d*\w*\]',
        'العوامل': r'(?:\+=|-=|\*=|/=|<=|>=|==|!=|&&|\|\||%=|<<|>>|>>=|<<=|&=|\|=|\^=|\+\+|--|\+|-|\*|/|%|<|>|&|\||\^|!|~|=)',
        'الدوال': r'\b(?:\w+\s+)+\w+\s*\([^)]*\)\s*(?:const)?\s*(?:{[^}]*})?'
    }
    results = {}

    # Extract declarations for each pattern
    for key, pattern in declaration_patterns.items():
        results[key] = len(re.findall(pattern, code))
    # Remove keys with value 0
    keys_to_remove = [key for key, value in results.items() if value == 0]
    for key in keys_to_remove:
        del results[key]
    return results

In [109]:
def find_general_concepts(code):
    code_without_comments = remove_comments(code)
    code_without_strings = re.sub(r'\".*?\"', '', code_without_comments)
    new_code = remove_main_function(code_without_strings)

    concepts_detected = detect_concept(new_code)
    general_concepts_keywords_count = pd.DataFrame(concepts_detected.items(), columns=['generalConcept', 'count'])
    concepts_dict = {k: 1 if k in concepts_detected else 0 for k in concepts_level}
    keys, values = zip(*concepts_dict.items())

    general_concepts_keywords_count['concepts_level'] = general_concepts_keywords_count['generalConcept'].apply(lambda x: concepts_level.get(x))

    total_keywords_count = general_concepts_keywords_count['count'].sum()

    general_concepts = general_concepts_keywords_count['generalConcept'].unique().tolist()
    general_concepts_keywords_count['code_difficulty'] = (general_concepts_keywords_count['concepts_level'] / 4)  * (general_concepts_keywords_count['count'] / total_keywords_count) *100

    return general_concepts, (general_concepts_keywords_count.code_difficulty.sum()), values


In [110]:
tutorial['generalConcepts'], tutorial['tutorial_difficulty'], tutorial[list(keys)] = zip(*tutorial['Source_code'].apply(find_general_concepts))
tutorial

,Number_Class,Source_code,الأساسيات,أنواع البيانات,المتغيرات,التعامل مع الأعداد,التعامل مع النصوص,العوامل,المصفوفات,الدوال,الحلقات,الشروط,generalConcepts,tutorial_difficulty
0,1A,#include <iostream>\r\nusing namespace std;\r\...,1,1,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",71.296296
1,282A,#include <iostream>\r\nusing namespace std;\r\...,1,1,0,0,0,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",71.969697
2,4A,#include <iostream>\r\nusing namespace std;\r\...,1,1,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",66.666667
3,158A,#include <iostream>\r\n#include <algorithm>\r\...,1,1,0,0,0,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",70.714286
4,118A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",74.107143
5,71A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,1,0,0,1,1,0,1,1,"[الحلقات, الشروط, المتغيرات, أنواع البيانات, ا...",72.074468
6,112A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,1,1,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",75.263158
7,281A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,1,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, المصفوفات,...",72.222222
8,236A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,1,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",73.376623
9,486A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",71.428571


In [111]:
minimum = tutorial.tutorial_difficulty.min()
maximum = tutorial.tutorial_difficulty.max()
tutorial['tutorial_difficulty'] = ((tutorial['tutorial_difficulty'] - minimum) / (maximum - minimum))*100
tutorial.reset_index(drop=True)

,Number_Class,Source_code,الأساسيات,أنواع البيانات,المتغيرات,التعامل مع الأعداد,التعامل مع النصوص,العوامل,المصفوفات,الدوال,الحلقات,الشروط,generalConcepts,tutorial_difficulty
0,1A,#include <iostream>\r\nusing namespace std;\r\...,1,1,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",68.919435
1,282A,#include <iostream>\r\nusing namespace std;\r\...,1,1,0,0,0,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",74.195564
2,4A,#include <iostream>\r\nusing namespace std;\r\...,1,1,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",32.646048
3,158A,#include <iostream>\r\n#include <algorithm>\r\...,1,1,0,0,0,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",64.359352
4,118A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",90.942563
5,71A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,1,0,0,1,1,0,1,1,"[الحلقات, الشروط, المتغيرات, أنواع البيانات, ا...",75.016451
6,112A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,1,1,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",100.000000
7,281A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,1,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, المصفوفات,...",76.174112
8,236A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,1,1,1,0,1,1,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",85.218905
9,486A,#include<iostream>\r\n#include<map>\r\n#includ...,1,1,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",69.955817


In [112]:
_, students_problemset['student_code_difficulty'], _ = zip(*students_problemset['Source_code'].apply(find_general_concepts))
students_problemset

,When,Who,Problem Name,Number,Class,Number_Class,A,B,C,D,...,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,Problem Class A,Problem Class B,previous_problems_count,student_code_difficulty
0,2021/10/18 19:29,Mohammed--alhefawe,Watermelon,4,A,4A,0,0,0,0,...,0,0,0,0,0,0,1,0,0,65.625000
1,2021/10/18 21:02,Mohammed--alhefawe,WayTooLongWords,71,A,71A,1,0,0,0,...,0,0,0,0,1,0,1,0,1,67.424242
2,2021/10/19 01:59,Mohammed--alhefawe,Bit++,282,A,282A,2,0,0,0,...,0,0,0,0,0,0,1,0,2,66.935484
3,2022/05/16 18:53,Mohammed--alhefawe,Team,231,A,231A,3,0,0,0,...,0,0,0,0,0,0,1,0,3,69.285714
4,2022/05/16 19:47,Mohammed--alhefawe,TheatreSquare,1,A,1A,4,0,0,0,...,0,0,0,0,0,0,1,0,4,65.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,2020/05/10 06:59,RamaRehawi,WrongSubtraction,977,A,977A,32,24,3,4,...,0,0,0,0,0,0,1,0,65,69.736842
2778,2020/05/10 07:05,RamaRehawi,Elephant,617,A,617A,33,24,3,4,...,0,0,0,0,0,0,1,0,66,65.384615
2779,2020/05/10 07:49,RamaRehawi,NearlyLuckyNumber,110,A,110A,34,24,3,4,...,0,0,0,0,0,0,1,0,67,72.619048
2780,2020/05/10 08:03,RamaRehawi,Translation,41,A,41A,35,24,3,4,...,0,0,0,0,1,0,1,0,68,71.428571


In [113]:
students_problemset['student_code_difficulty'] = ((students_problemset['student_code_difficulty'] - minimum) / (maximum - minimum))*100
students_problemset.reset_index(drop=True)

,When,Who,Problem Name,Number,Class,Number_Class,A,B,C,D,...,Tag_numbertheory,Tag_shortestpaths,Tag_sortings,Tag_specialproblem,Tag_strings,Tag_twopointers,Problem Class A,Problem Class B,previous_problems_count,student_code_difficulty
0,2021/10/18 19:29,Mohammed--alhefawe,Watermelon,4,A,4A,0,0,0,0,...,0,0,0,0,0,0,1,0,0,24.484536
1,2021/10/18 21:02,Mohammed--alhefawe,WayTooLongWords,71,A,71A,1,0,0,0,...,0,0,0,0,1,0,1,0,1,38.581693
2,2021/10/19 01:59,Mohammed--alhefawe,Bit++,282,A,282A,2,0,0,0,...,0,0,0,0,0,0,1,0,2,34.752245
3,2022/05/16 18:53,Mohammed--alhefawe,Team,231,A,231A,3,0,0,0,...,0,0,0,0,0,0,1,0,3,53.166421
4,2022/05/16 19:47,Mohammed--alhefawe,TheatreSquare,1,A,1A,4,0,0,0,...,0,0,0,0,0,0,1,0,4,23.318606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,2020/05/10 06:59,RamaRehawi,WrongSubtraction,977,A,977A,32,24,3,4,...,0,0,0,0,0,0,1,0,65,56.701031
2778,2020/05/10 07:05,RamaRehawi,Elephant,617,A,617A,33,24,3,4,...,0,0,0,0,0,0,1,0,66,22.601110
2779,2020/05/10 07:49,RamaRehawi,NearlyLuckyNumber,110,A,110A,34,24,3,4,...,0,0,0,0,0,0,1,0,67,79.283260
2780,2020/05/10 08:03,RamaRehawi,Translation,41,A,41A,35,24,3,4,...,0,0,0,0,1,0,1,0,68,69.955817


In [114]:
students_problemset.columns

Index(['When', 'Who', 'Problem Name', 'Number', 'Class', 'Number_Class', 'A',
       'B', 'C', 'D', 'J', 'E', 'L', 'K', 'G', 'H', 'B1', 'B2', 'G1', 'F', 'I',
       'M', 'B3', 'D1', 'D2', 'Y', 'X', 'Z', 'R', 'N', 'L1', 'L2', 'C1', 'F1',
       'E1', 'C2', '100', 'F2', 'E2', 'Q', 'P', 'A1', 'A2', 'O', 'U', 'W', 'T',
       'G3', 'G2', 'F3', '1', 'S', '101', 'H1', 'H2', '123', 'Source_code',
       'Difficulty', 'Solved', 'Tag_binarysearch', 'Tag_bitmasks',
       'Tag_bruteforce', 'Tag_constructivealgorithms', 'Tag_datastructures',
       'Tag_dp', 'Tag_expressionparsing', 'Tag_graphmatchings', 'Tag_greedy',
       'Tag_hashing', 'Tag_implementation', 'Tag_math', 'Tag_numbertheory',
       'Tag_shortestpaths', 'Tag_sortings', 'Tag_specialproblem',
       'Tag_strings', 'Tag_twopointers', 'Problem Class A', 'Problem Class B',
       'previous_problems_count', 'student_code_difficulty'],
      dtype='object')

In [115]:
tutorial.columns

Index(['Number_Class', 'Source_code', 'الأساسيات', 'أنواع البيانات',
       'المتغيرات', 'التعامل مع الأعداد', 'التعامل مع النصوص', 'العوامل',
       'المصفوفات', 'الدوال', 'الحلقات', 'الشروط', 'generalConcepts',
       'tutorial_difficulty'],
      dtype='object')

In [120]:
tutorial.rename(columns = {"Source_code":"tutorial_code"}, inplace = True)

students_problemset.rename(columns = {"student_code_difficulty":"y", "Who": "Student", "previous_problems_count":"previous_solved_problems_count"}, inplace = True)
students_problemset.rename(columns = {"Difficulty":"problem_difficulty", "Solved":"solving_times"}, inplace = True)


students_problemset.drop(columns = ['When', 'Number', 'Class', 'Source_code'], inplace = True)

In [121]:
students_problemset.columns

Index(['Student', 'Problem Name', 'Number_Class', 'A', 'B', 'C', 'D', 'J', 'E',
       'L', 'K', 'G', 'H', 'B1', 'B2', 'G1', 'F', 'I', 'M', 'B3', 'D1', 'D2',
       'Y', 'X', 'Z', 'R', 'N', 'L1', 'L2', 'C1', 'F1', 'E1', 'C2', '100',
       'F2', 'E2', 'Q', 'P', 'A1', 'A2', 'O', 'U', 'W', 'T', 'G3', 'G2', 'F3',
       '1', 'S', '101', 'H1', 'H2', '123', 'problem_difficulty',
       'solving_times', 'Tag_binarysearch', 'Tag_bitmasks', 'Tag_bruteforce',
       'Tag_constructivealgorithms', 'Tag_datastructures', 'Tag_dp',
       'Tag_expressionparsing', 'Tag_graphmatchings', 'Tag_greedy',
       'Tag_hashing', 'Tag_implementation', 'Tag_math', 'Tag_numbertheory',
       'Tag_shortestpaths', 'Tag_sortings', 'Tag_specialproblem',
       'Tag_strings', 'Tag_twopointers', 'Problem Class A', 'Problem Class B',
       'previous_solved_problems_count', 'y'],
      dtype='object')

In [122]:
tutorial.columns

Index(['Number_Class', 'tutorial_code', 'الأساسيات', 'أنواع البيانات',
       'المتغيرات', 'التعامل مع الأعداد', 'التعامل مع النصوص', 'العوامل',
       'المصفوفات', 'الدوال', 'الحلقات', 'الشروط', 'generalConcepts',
       'tutorial_difficulty'],
      dtype='object')

In [123]:
final_df = students_problemset.merge(tutorial, on = ['Number_Class'], how = 'inner')
final_df

,Student,Problem Name,Number_Class,A,B,C,D,J,E,L,...,المتغيرات,التعامل مع الأعداد,التعامل مع النصوص,العوامل,المصفوفات,الدوال,الحلقات,الشروط,generalConcepts,tutorial_difficulty
0,Mohammed--alhefawe,Watermelon,4A,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",32.646048
1,OmranMk,Watermelon,4A,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",32.646048
2,RetiringNextCentury,Watermelon,4A,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",32.646048
3,HASSAN_GH,Watermelon,4A,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",32.646048
4,Brre.Hemo,Watermelon,4A,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,"[الشروط, أنواع البيانات, الأساسيات, العوامل]",32.646048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,Nour-Aldeen,GravityFlip,405A,85,38,21,2,0,1,0,...,1,0,0,1,1,1,1,1,"[الحلقات, الشروط, المتغيرات, أنواع البيانات, ا...",64.166370
2778,0xessam,GravityFlip,405A,41,7,2,0,0,0,1,...,1,0,0,1,1,1,1,1,"[الحلقات, الشروط, المتغيرات, أنواع البيانات, ا...",64.166370
2779,ghaith.clash88,GravityFlip,405A,36,7,1,0,0,0,0,...,1,0,0,1,1,1,1,1,"[الحلقات, الشروط, المتغيرات, أنواع البيانات, ا...",64.166370
2780,sarah.m.akkad,GravityFlip,405A,3,0,0,0,0,0,0,...,1,0,0,1,1,1,1,1,"[الحلقات, الشروط, المتغيرات, أنواع البيانات, ا...",64.166370


In [124]:
final_df.columns

Index(['Student', 'Problem Name', 'Number_Class', 'A', 'B', 'C', 'D', 'J', 'E',
       'L', 'K', 'G', 'H', 'B1', 'B2', 'G1', 'F', 'I', 'M', 'B3', 'D1', 'D2',
       'Y', 'X', 'Z', 'R', 'N', 'L1', 'L2', 'C1', 'F1', 'E1', 'C2', '100',
       'F2', 'E2', 'Q', 'P', 'A1', 'A2', 'O', 'U', 'W', 'T', 'G3', 'G2', 'F3',
       '1', 'S', '101', 'H1', 'H2', '123', 'problem_difficulty',
       'solving_times', 'Tag_binarysearch', 'Tag_bitmasks', 'Tag_bruteforce',
       'Tag_constructivealgorithms', 'Tag_datastructures', 'Tag_dp',
       'Tag_expressionparsing', 'Tag_graphmatchings', 'Tag_greedy',
       'Tag_hashing', 'Tag_implementation', 'Tag_math', 'Tag_numbertheory',
       'Tag_shortestpaths', 'Tag_sortings', 'Tag_specialproblem',
       'Tag_strings', 'Tag_twopointers', 'Problem Class A', 'Problem Class B',
       'previous_solved_problems_count', 'y', 'tutorial_code', 'الأساسيات',
       'أنواع البيانات', 'المتغيرات', 'التعامل مع الأعداد',
       'التعامل مع النصوص', 'العوامل', 'المصفوفات',

In [125]:
final_df.drop(columns = ['generalConcepts', 'tutorial_code', 'Number_Class'], inplace = True)

In [126]:
final_df.columns

Index(['Student', 'Problem Name', 'A', 'B', 'C', 'D', 'J', 'E', 'L', 'K', 'G',
       'H', 'B1', 'B2', 'G1', 'F', 'I', 'M', 'B3', 'D1', 'D2', 'Y', 'X', 'Z',
       'R', 'N', 'L1', 'L2', 'C1', 'F1', 'E1', 'C2', '100', 'F2', 'E2', 'Q',
       'P', 'A1', 'A2', 'O', 'U', 'W', 'T', 'G3', 'G2', 'F3', '1', 'S', '101',
       'H1', 'H2', '123', 'problem_difficulty', 'solving_times',
       'Tag_binarysearch', 'Tag_bitmasks', 'Tag_bruteforce',
       'Tag_constructivealgorithms', 'Tag_datastructures', 'Tag_dp',
       'Tag_expressionparsing', 'Tag_graphmatchings', 'Tag_greedy',
       'Tag_hashing', 'Tag_implementation', 'Tag_math', 'Tag_numbertheory',
       'Tag_shortestpaths', 'Tag_sortings', 'Tag_specialproblem',
       'Tag_strings', 'Tag_twopointers', 'Problem Class A', 'Problem Class B',
       'previous_solved_problems_count', 'y', 'الأساسيات', 'أنواع البيانات',
       'المتغيرات', 'التعامل مع الأعداد', 'التعامل مع النصوص', 'العوامل',
       'المصفوفات', 'الدوال', 'الحلقات', 'الشروط', '

In [127]:
final_df.to_csv('final_df.csv', encoding = 'utf-8', index = False)

# The final dataset has the problem name, its class, tutorial code difficulty, general concepts in tutorial code, problem difficulty, problem tags, and its solving times on codeforces as well as, student name, his previous solved problems counts, along with their classes, and the y value which represents his code difficulty

### Baseline: Simple Linear Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


linreg = LinearRegression().fit(X_train, y_train)

In [ ]:
print('Codeforces dataset')
print('linear model intercept: {}'
     .format(linreg.intercept_))
print('linear model coeff:\n{}'
     .format(linreg.coef_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))

### Dummy regressors

# FOR DELETION (WE CAN TRY APPLYING MULTIPLE OPTIONS)
strategy parameter options:
- mean : predicts the mean of the training targets.
- median : predicts the median of the training targets.
- quantile : predicts a user-provided quantile of the training target values (e.g. value at the 75th percentile)
- constant : predicts a constant user-provided value.

In [ ]:
# - Alternative metrics include:
#     - mean_absolute_error (absolute difference of target & predicted values)
#     - mean_squared_error (squared difference of target & predicted values)
    
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train, y_train)

y_predict = linreg.predict(X_test)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test)

print('Linear model, coefficients: ', linreg.coef_)
print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(y_test, 
                                                                     y_predict_dummy_mean)))
print("Mean squared error (linear model): {:.2f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score (dummy): {:.2f}".format(r2_score(y_test, y_predict_dummy_mean)))
print("r2_score (linear model): {:.2f}".format(r2_score(y_test, y_predict)))

# Plot outputs
plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, y_predict, color='green', linewidth=2)
plt.plot(X_test, y_predict_dummy_mean, color='red', linestyle = 'dashed', 
         linewidth=2, label = 'dummy')

plt.show()

### Scaling 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)

### Ridge regression

In [ ]:
from sklearn.linear_model import Ridge

linridge = Ridge(alpha=20.0).fit(X_train, y_train)

print('Codeforces dataset')
print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

#### Ridge regression with regularization parameter: alpha

In [ ]:
# Do NOT forget to discuss several alpha values

print('Ridge regression: effect of alpha regularization parameter\n')
all_alphas = [0, 1, 10, 20, 50, 100, 1000]
all_r2_train = []
all_r2_test = []
for this_alpha in all_alphas:
    linridge = Ridge(alpha = this_alpha).fit(X_train_scaled, y_train)
    r2_train = linridge.score(X_train_scaled, y_train)
    all_r2_train.append(r2_train)
    r2_test = linridge.score(X_test_scaled, y_test)
    all_r2_test.append(r2_test)
    num_coeff_bigger = np.sum(abs(linridge.coef_) > 1.0)
    
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
r-squared training: {:.2f}, r-squared test: {:.2f}\n'
         .format(this_alpha, num_coeff_bigger, r2_train, r2_test))
    

In [ ]:
plt.figure(figsize=(5,4))
alphas = all_alphas
plt.scatter(alphas, all_r2_train, marker= 'o', s=50, alpha=0.8)
plt.scatter(alphas, all_r2_test, marker= 'x', s=50, alpha=0.8)
plt.title('R2 Error Over Different Alphas')
plt.xlabel('alpha values')
plt.ylabel('r2 error')
plt.show()

### Lasso regression

In [ ]:
print('Lasso regression: effect of alpha regularization\n\
parameter on number of features kept in final model\n')

linlasso = Lasso(alpha, max_iter = 10000).fit(X_train, y_train)

print('Codeforces dataset')
print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('R-squared score (training): {:.3f}'
     .format(linlasso.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linlasso.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))


#### Lasso regression with regularization parameter: alpha

In [ ]:
print('Lasso regression: effect of alpha regularization\n\
parameter on number of features kept in final model\n')
all_alphas = [0.5, 1, 2, 3, 5, 10, 20, 50]
all_r2_train = []
all_r2_test = []
for alpha in all_alphas:
    linlasso = Lasso(alpha, max_iter = 10000).fit(X_train_scaled, y_train)
    r2_train = linlasso.score(X_train_scaled, y_train)
    r2_test = linlasso.score(X_test_scaled, y_test)
    all_r2_train.append(r2_train)
    all_r2_test.append(r2_test)
    
    print('Alpha = {:.2f}\nFeatures kept: {}, r-squared training: {:.2f}, \
r-squared test: {:.2f}\n'
         .format(alpha, np.sum(linlasso.coef_ != 0), r2_train, r2_test))

In [ ]:
plt.figure(figsize=(5,4))
alphas = all_alphas
plt.scatter(alphas, all_r2_train, marker= 'o', s=50, alpha=0.8)
plt.scatter(alphas, all_r2_test, marker= 'x', s=50, alpha=0.8)
plt.title('R2 Error Over Different Alphas')
plt.xlabel('alpha values')
plt.ylabel('r2 error')
plt.show()

### KNN

# TO BE DELETED

With regression, **what we do is instead of taking a majority vote, we don't have class values here as targets, we have continuous values. So we can average these  target values. And if we do that, we find that the output, when the query point is this X-value, is going to be the average of the y-values of the three nearest training points**.


**KNeighborsRegressor: important parameters**
- Model complexity: n_neighbors: number of nearest neighbors (k) to consider–Default  = 5
- Model fitting: metric:  distance function between data points–Default: Minkowski distance with power parameter p = 2 (Euclidean)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X_train, y_train)

print(knnreg.predict(X_test))
print('R-squared test score: {:.3f}'
     .format(knnreg.score(X_test, y_test)))

In [ ]:
fig, subaxes = plt.subplots(1, 2, figsize=(8,4))
X_predict_input = np.linspace(-3, 3, 50).reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X_R1[0::5], y_R1[0::5], random_state = 0)

for thisaxis, K in zip(subaxes, [1, 3]):
    knnreg = KNeighborsRegressor(n_neighbors = K).fit(X_train, y_train)
    y_predict_output = knnreg.predict(X_predict_input)
    thisaxis.set_xlim([-2.5, 0.75])
    thisaxis.plot(X_predict_input, y_predict_output, '^', markersize = 10,
                 label='Predicted', alpha=0.8)
    thisaxis.plot(X_train, y_train, 'o', label='True Value', alpha=0.8)
    thisaxis.set_xlabel('Input feature')
    thisaxis.set_ylabel('Target value')
    thisaxis.set_title('KNN regression (K={})'.format(K))
    thisaxis.legend()
plt.tight_layout()

### Regression model complexity as a function of K


In [ ]:
# plot k-NN regression on sample dataset for different values of K
fig, subaxes = plt.subplots(5, 1, figsize=(5,20))
X_predict_input = np.linspace(-3, 3, 500).reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X_R1, y_R1,
                                                   random_state = 0)

for thisaxis, K in zip(subaxes, [1, 3, 7, 15, 55]):
    knnreg = KNeighborsRegressor(n_neighbors = K).fit(X_train, y_train)
    y_predict_output = knnreg.predict(X_predict_input)
    train_score = knnreg.score(X_train, y_train)
    test_score = knnreg.score(X_test, y_test)
    thisaxis.plot(X_predict_input, y_predict_output)
    thisaxis.plot(X_train, y_train, 'o', alpha=0.9, label='Train')
    thisaxis.plot(X_test, y_test, '^', alpha=0.9, label='Test')
    thisaxis.set_xlabel('Input feature')
    thisaxis.set_ylabel('Target value')
    thisaxis.set_title('KNN Regression (K={})\n\
Train $R^2 = {:.3f}$,  Test $R^2 = {:.3f}$'
                      .format(K, train_score, test_score))
    thisaxis.legend()
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

# TO BE DELETED
Starting on the left when k = 1, the regression model fits the training data perfectly with a r-squared score of 1.0. But it's very bad at predicting the target values for new data samples, as reflected in the r-squared test score of only 0.155. As the value of k increases, which we can see acts to smooth out these local variations to capture more of the global trend. 

Again the training set score drops, but the model gets better at generalizing to new data and the test score goes up as K increases. Finally in this series, the model with k = 15 has the best test set performance, with an r-squared score of 0.485. Increasing k much further however to k = 55, results in both the training and test set scores dropping back down to lower levels, as the model now starts to under-fit. In other words, it's too simple to do well, even on the training data. 

#### When to use KNN
The pro's of the nearest neighbor approach are that it's simple and easy to understand why a particular prediction is made. A k-nearest neighbor approach can be a reasonable baseline against what you can compare more sophisticated methods. 

When the training data has many instances, or each instance has lots of features, this can really slow down the performance of a k-nearest neighbors model. So in general, if your data set has hundreds or thousands of features, you should consider alternatives to k-nearest neighbors models, especially if your data is sparse. Meaning that each instance has lots of features, but most of them are zero. 

In [ ]:
- Alternative metrics include:
    - mean_absolute_error (absolute difference of target & predicted values)
    - mean_squared_error (squared difference of target & predicted values)